# 빅콘테스트 편성 계획을 위한 데이터 분석
> 편성 계획을 세우기 위해서는 데이터를 좀 더 자세히 분석해볼 필요가 있다. 어떤 상품이 어떤 시기에, 그리고 어떤 시간에 가장 잘 팔릴지를 고려해 편성표를 구성해야 한다. 그를 위해서는 기준을 세워 데이터를 분석해볼 필요가 있다. 우선 가장 간단한, 상품군을 기준으로 데이터를 분석하여 가장 잘 팔리는 시간대를 찾는다.

- 데이터프레임을 상품군별로 분리한다. 
- 취급액이 가장 높은 상품을 찾고, 시간대를 확인한다. 

# 데이터 상품군별로 분리하기
> 데이터 상품군 일람은 아래와 같다. 
                    
- 0: 주방
- 1: 가전
- 2: 의류
- 3: 속옷
- 4: 잡화
- 5: 농수축
- 6: 생활용품
- 7: 가구
- 8: 이미용
- 9: 건강기능
- 10: 침구

In [84]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np
import pickle
import sys

In [308]:
# 주문량 column 추가 
def add_column(df):
    df['주문량'] = df['취급액'] / df['판매단가']
    df = df.astype({'주문량' : int})
    return df

# 데이터프레임 인덱싱 함수(top 100~500)
def index_dataframe(df, num):
    df = df[:num-1]
    return df

# 특정 column 중복 제거 후 list화 함수
def column_set_to_list(df, column_name):
    column_li = list(set(df[column_name]))
    return column_li

# list 안에 포함되는 모든 열들을 병합, 데이터프레임 생성
def filter_dataframe_for_list(df,column_name, li):
    df_concat = df[df[column_name] == li[0]]
    for i in range(1, len(li)-1):
        new_raw = df[df[column_name] == li[i]]
        df_concat = pd.concat([df_concat, new_raw])

    return df_concat

# 데이터를 특정 column 기준으로 내림차순한 후 첫 줄을 반환하는 함수
def golden_time_for_product(df, column_name):
    df = df.sort_values(by=column_name, ascending=False)
    return df[:1]


In [85]:
# 전처리한 데이터 불러오기
with open('dataframe_2.pkl', 'rb') as f:
    load_df=pickle.load(f)
    
load_df

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,성별 포함,년,월,일자,시,분,초
1,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,2,39900,2099000,1,2019,1,1,6,0,0
2,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,2,39900,4371000,2,2019,1,1,6,0,0
3,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,2,39900,3262000,1,2019,1,1,6,20,0
4,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,2,39900,6955000,2,2019,1,1,6,20,0
5,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,2,39900,6672000,1,2019,1,1,6,40,0
6,2019-01-01 06:40:00,20.0,100346,201079,테이트 여성 셀린니트3종,2,39900,9337000,2,2019,1,1,6,40,0
7,2019-01-01 07:00:00,20.0,100305,200974,오모떼 레이스 파운데이션 브라,3,59000,6819000,2,2019,1,1,7,0,0
8,2019-01-01 07:20:00,20.0,100305,200974,오모떼 레이스 파운데이션 브라,3,59000,15689000,2,2019,1,1,7,20,0
9,2019-01-01 07:40:00,20.0,100305,200974,오모떼 레이스 파운데이션 브라,3,59000,25370000,2,2019,1,1,7,40,0
10,2019-01-01 08:00:00,20.0,100808,202377,CERINI by PAT 남성 소프트 기모 릴렉스팬츠,2,59900,16133000,1,2019,1,1,8,0,0


In [86]:
# 상품군 숫자 확인
load_df['상품군'].value_counts()

0     6604
1     6479
2     4340
3     3919
4     3891
5     3884
6     2877
7     2614
8     1305
9      790
10     669
Name: 상품군, dtype: int64

In [87]:
# 상품군 기준으로 데이터 분리하기
mod=sys.modules[__name__]

for i in range(11):
    setattr(mod, 'df_{}'.format(i), load_df[load_df['상품군']==i])

print('데이터 분리 완료')

데이터 분리 완료


In [88]:
# 데이터 확인
df_2['상품군'].value_counts()

2    4340
Name: 상품군, dtype: int64

# 상품군별 분석
## 데이터별 취급액, 주문량 기준으로 sort
> 취급액(판매량) 기준으로 sort했을 때와 주문량으로 sort했을 때의 인기 상품군 순위에 차이가 존재할 수 있다. 

- 주문량
    - 절대적인 주문량을 기준으로 하므로 시청자들의 선호도와 연관성이 깊고, 시청자가 부담없어하는 가격대를 파악할 수 있다.  

<br>

- 취급액
    - 절대 판매액이 기준인 만큼 기본 가격대가 높은 상품들이 상위에 포진되기 쉽고, 시청자들의 선호도를 따지기에는 주문량 쪽이 조금은 더 나은 기준이라고 할 수 있다. 그러나 시간과 함께 고려하여 보다 나은 매출을 이끌어낼 수 있다. 
    
    
<br>
<br>
따라서 이 두 가지 기준 모두를 고려하여 시청률이 높은 시간대에 배치할 상품들을 선정한다.

## 주방

### 취급액

In [89]:
#'주방'                     
df_0_sold = df_0.sort_values(by=['취급액'], ascending=False)

In [374]:
# 인기상품 top 100(취급액 기준)
df_0_sold = df_0_sold.reset_index(drop=True)
top_100_0 = df_0_sold.loc[:99]
top_100_0

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,성별 포함,년,월,일자,시,분,초
0,2019-11-28 16:20:00,20.0,100255,200869,한일 대용량 스텐 분쇄믹서기,0,109800,225084000,0,2019,11,28,16,20,0
1,2019-12-29 16:00:00,20.0,100724,202115,IH 옛 가마솥 세트,0,62000,167261000,0,2019,12,29,16,0,0
2,2019-11-17 11:00:00,20.0,100255,200869,한일 대용량 스텐 분쇄믹서기,0,109800,142764000,0,2019,11,17,11,0,0
3,2019-12-29 13:00:00,20.0,100638,201956,램프쿡 자동회전냄비,0,109000,141159000,0,2019,12,29,13,0,0
4,2019-12-28 09:00:00,20.0,100264,200878,셀렉프로 뚝배기 전기밥솥(멀티쿠커),0,79000,128382000,0,2019,12,28,9,0,0
5,2019-11-24 17:00:00,20.0,100837,202467,무이자 쿠쿠전기밥솥 10인용(CRP-QS107FG/FS),0,218000,119573000,0,2019,11,24,17,0,0
6,2019-09-22 13:40:00,20.0,100255,200868,한일 대용량 스텐 분쇄믹서기,0,109800,110536000,0,2019,9,22,13,40,0
7,2019-03-10 23:00:00,20.0,100770,202250,무이자 에지리 스텐큐브 프라이팬,0,179000,109943000,0,2019,3,10,23,0,0
8,2019-09-07 15:40:00,20.0,100255,200868,한일 대용량 스텐 분쇄믹서기,0,109800,109365000,0,2019,9,7,15,40,0
9,2019-11-18 15:40:00,20.0,100255,200869,한일 대용량 스텐 분쇄믹서기,0,109800,108086000,0,2019,11,18,15,40,0


In [123]:
# 각 상품명별, 마더코드별 중복 삭제
# 상품명
name_li_0 = list(set(top_100_0['상품명']))
# print(len(name_li_0))
name_li_0 # 상품명 100개 중 75개가 겹침

25


['무이자 쿠쿠전기밥솥 10인용(CRP-QS107FG/FS)',
 '일시불 스위스밀리터리 에어프라이어',
 '해피콜  IH 진공 스텐 냄비 에디션',
 '무이자 스위스밀리터리 에어프라이어',
 '일시불 쿠쿠전기밥솥 10인용 (QS)',
 '키친아트 스마트뷰 스텐밀폐용기 20종',
 '키친플라워 대용량 디지털 에어프라이어 5L',
 '한일 대용량 스텐 분쇄믹서기',
 '아이넥스 프리미엄 올스텐 싱크선반',
 '안타고 만능 가스그릴',
 '일시불 에지리 스텐큐브 프라이팬',
 '무이자 에지리 스텐큐브 프라이팬',
 '셀렉프로 뚝배기 전기밥솥(멀티쿠커)',
 '노와 스텐 대용량에어프라이어 4L',
 '[세라맥스] 포터블 원적외선 전기렌인지',
 '일시불쿠첸압력밥솥 10인용',
 '초특가  파뷔에 팬지 대형 후라이팬 세트',
 '에코라믹 컬링스톤프라이팬세트',
 '해피콜엑슬림블렌더',
 '파뷔에 항균도마 1+1+칼세트',
 '에코라믹 통주물 스톤 냄비세트',
 '램프쿡 자동회전냄비',
 'IH 옛 가마솥 세트',
 '파뷔에 항균도마 1+1세트',
 '일시불 매직쉐프 10리터 듀얼쿡 에어프라이어']

In [126]:
# 마더코드별 
mothercode_0 = list(set(top_100_0['마더코드']))
mothercode_0 # 100개 중 80개의 마더코드가 중복

[100231,
 100366,
 100240,
 100509,
 100638,
 100255,
 100770,
 100517,
 100264,
 100520,
 100657,
 100673,
 100683,
 100448,
 100066,
 100837,
 100724,
 100088,
 100475,
 100478]

### 주문량

In [91]:
# 주문량 기준
df_0 = add_column(df_0)
df_0 = df_0.sort_values(by=['주문량'], ascending=False)
df_0

C:\Users\CUCHO\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,성별 포함,년,월,일자,시,분,초,주문량
38086,2019-12-29 16:00:00,20.0,100724,202115,IH 옛 가마솥 세트,0,62000,167261000,0,2019,12,29,16,0,0,2697
34907,2019-11-28 16:20:00,20.0,100255,200869,한일 대용량 스텐 분쇄믹서기,0,109800,225084000,0,2019,11,28,16,20,0,2049
38059,2019-12-29 08:00:00,20.0,100236,200806,벨라홈 스마트 멀티포트 1+1 세트,0,39800,68162000,0,2019,12,29,8,0,0,1712
23,2019-01-01 12:40:00,20.0,100088,200236,에코라믹 통주물 스톤 냄비세트,0,60900,99235000,0,2019,1,1,12,40,0,1629
37962,2019-12-28 09:00:00,20.0,100264,200878,셀렉프로 뚝배기 전기밥솥(멀티쿠커),0,79000,128382000,0,2019,12,28,9,0,0,1625
14642,2019-05-18 09:40:00,20.0,100478,201453,파뷔에 항균도마 1+1세트,0,49800,79405000,0,2019,5,18,9,40,0,1594
17636,2019-06-15 08:30:00,30.0,100478,201452,파뷔에 항균도마 1+1+칼세트,0,49800,79181000,0,2019,6,15,8,30,0,1589
1134,2019-01-13 12:40:00,20.0,100088,200236,에코라믹 통주물 스톤 냄비세트,0,60900,96399000,0,2019,1,13,12,40,0,1582
38258,2019-12-31 17:40:00,20.0,100236,200806,벨라홈 스마트 멀티포트 1+1 세트,0,39800,60256000,0,2019,12,31,17,40,0,1513
7347,2019-03-16 11:40:00,20.0,100088,200236,에코라믹 통주물 스톤 냄비세트,0,60900,91090000,0,2019,3,16,11,40,0,1495


In [376]:
# 인기상품 top 100(주문량 기준)
top_100_00 = df_0[:100]
top_100_00

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,성별 포함,년,월,일자,시,분,초,주문량
38086,2019-12-29 16:00:00,20.0,100724,202115,IH 옛 가마솥 세트,0,62000,167261000,0,2019,12,29,16,0,0,2697
34907,2019-11-28 16:20:00,20.0,100255,200869,한일 대용량 스텐 분쇄믹서기,0,109800,225084000,0,2019,11,28,16,20,0,2049
38059,2019-12-29 08:00:00,20.0,100236,200806,벨라홈 스마트 멀티포트 1+1 세트,0,39800,68162000,0,2019,12,29,8,0,0,1712
23,2019-01-01 12:40:00,20.0,100088,200236,에코라믹 통주물 스톤 냄비세트,0,60900,99235000,0,2019,1,1,12,40,0,1629
37962,2019-12-28 09:00:00,20.0,100264,200878,셀렉프로 뚝배기 전기밥솥(멀티쿠커),0,79000,128382000,0,2019,12,28,9,0,0,1625
14642,2019-05-18 09:40:00,20.0,100478,201453,파뷔에 항균도마 1+1세트,0,49800,79405000,0,2019,5,18,9,40,0,1594
17636,2019-06-15 08:30:00,30.0,100478,201452,파뷔에 항균도마 1+1+칼세트,0,49800,79181000,0,2019,6,15,8,30,0,1589
1134,2019-01-13 12:40:00,20.0,100088,200236,에코라믹 통주물 스톤 냄비세트,0,60900,96399000,0,2019,1,13,12,40,0,1582
38258,2019-12-31 17:40:00,20.0,100236,200806,벨라홈 스마트 멀티포트 1+1 세트,0,39800,60256000,0,2019,12,31,17,40,0,1513
7347,2019-03-16 11:40:00,20.0,100088,200236,에코라믹 통주물 스톤 냄비세트,0,60900,91090000,0,2019,3,16,11,40,0,1495


In [125]:
# 상품명별, 마더코드별 중복 제거
# 상품명
name_li_00 = list(set(top_100_00['상품명']))
# print(len(name_li_00)) 
name_li_00 # 상품명 100개 중 88개가 중복

12


['셀렉프로 뚝배기 전기밥솥(멀티쿠커)',
 '벨라홈 스마트 멀티포트 1+1 세트',
 '파뷔에 항균도마 1+1+칼세트',
 '한일 대용량 스텐 분쇄믹서기',
 '초특가  파뷔에 팬지 대형 후라이팬 세트',
 '에코라믹 통주물 스톤 냄비세트',
 '램프쿡 자동회전냄비',
 'IH 옛 가마솥 세트',
 '일시불 클란츠 멀티쿡',
 '파뷔에 항균도마 1+1세트',
 '에코라믹 컬링스톤프라이팬세트',
 '세균싹 행주티슈']

In [129]:
# 마더코드별
mothercode_00 = list(set(top_100_00['마더코드']))
print(len(mothercode_00))
mothercode_00 # 마더코드 100개 중 90개 중복

10


[100483,
 100294,
 100264,
 100236,
 100724,
 100088,
 100638,
 100475,
 100478,
 100255]

### 두 기준 공통

In [235]:
# 두 기준 모두에 포함되는 상품명
popular_name_0 = list(set(name_li_0) & set(name_li_00))
popular_name_0

['셀렉프로 뚝배기 전기밥솥(멀티쿠커)',
 '파뷔에 항균도마 1+1+칼세트',
 '한일 대용량 스텐 분쇄믹서기',
 '초특가  파뷔에 팬지 대형 후라이팬 세트',
 '에코라믹 통주물 스톤 냄비세트',
 '램프쿡 자동회전냄비',
 'IH 옛 가마솥 세트',
 '파뷔에 항균도마 1+1세트',
 '에코라믹 컬링스톤프라이팬세트']

In [199]:
# 두 기준 모두에 해당되는 마더코드
set(mothercode_00) & set(mothercode_0)

{100088, 100255, 100264, 100475, 100478, 100638, 100724}

## 데이터프레임에서 판매 단가 비교
> 위에서 구한 상품군별 인기 상품을 데이터프레임 내에서 탐색하고, 판매단가를 비교하여 최적 상품을 찾아본다.

In [240]:
filter_df0 = filter_dataframe_for_list(load_df, '상품명', popular_name_0)
filter_df0.head(10)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,성별 포함,년,월,일자,시,분,초
37228,2019-12-20 08:20:00,20.0,100264,200878,셀렉프로 뚝배기 전기밥솥(멀티쿠커),0,79000,29415000,0,2019,12,20,8,20,0
37229,2019-12-20 08:40:00,20.0,100264,200878,셀렉프로 뚝배기 전기밥솥(멀티쿠커),0,79000,58490000,0,2019,12,20,8,40,0
37230,2019-12-20 09:00:00,20.0,100264,200878,셀렉프로 뚝배기 전기밥솥(멀티쿠커),0,79000,85353000,0,2019,12,20,9,0,0
37960,2019-12-28 08:20:00,20.0,100264,200878,셀렉프로 뚝배기 전기밥솥(멀티쿠커),0,79000,46738000,0,2019,12,28,8,20,0
37961,2019-12-28 08:40:00,20.0,100264,200878,셀렉프로 뚝배기 전기밥솥(멀티쿠커),0,79000,86252000,0,2019,12,28,8,40,0
37962,2019-12-28 09:00:00,20.0,100264,200878,셀렉프로 뚝배기 전기밥솥(멀티쿠커),0,79000,128382000,0,2019,12,28,9,0,0
16784,2019-06-07 08:00:00,30.0,100478,201452,파뷔에 항균도마 1+1+칼세트,0,49800,36015000,0,2019,6,7,8,0,0
16785,2019-06-07 08:30:00,30.0,100478,201452,파뷔에 항균도마 1+1+칼세트,0,49800,70355000,0,2019,6,7,8,30,0
17106,2019-06-10 09:00:00,30.0,100478,201452,파뷔에 항균도마 1+1+칼세트,0,49800,35161000,0,2019,6,10,9,0,0
17107,2019-06-10 09:30:00,30.0,100478,201452,파뷔에 항균도마 1+1+칼세트,0,49800,57050000,0,2019,6,10,9,30,0


In [278]:
# 상품명 기준으로 데이터 분리하기
mod=sys.modules[__name__]

for i in range(len(popular_name_0)):
    setattr(mod, 'refilter_df0_{}'.format(i), load_df[load_df['상품명']== popular_name_0[i]])

print('데이터 분리 완료')

데이터 분리 완료


In [279]:
# 상품별 데이터
refilter_df0_0

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,성별 포함,년,월,일자,시,분,초
37228,2019-12-20 08:20:00,20.0,100264,200878,셀렉프로 뚝배기 전기밥솥(멀티쿠커),0,79000,29415000,0,2019,12,20,8,20,0
37229,2019-12-20 08:40:00,20.0,100264,200878,셀렉프로 뚝배기 전기밥솥(멀티쿠커),0,79000,58490000,0,2019,12,20,8,40,0
37230,2019-12-20 09:00:00,20.0,100264,200878,셀렉프로 뚝배기 전기밥솥(멀티쿠커),0,79000,85353000,0,2019,12,20,9,0,0
37960,2019-12-28 08:20:00,20.0,100264,200878,셀렉프로 뚝배기 전기밥솥(멀티쿠커),0,79000,46738000,0,2019,12,28,8,20,0
37961,2019-12-28 08:40:00,20.0,100264,200878,셀렉프로 뚝배기 전기밥솥(멀티쿠커),0,79000,86252000,0,2019,12,28,8,40,0
37962,2019-12-28 09:00:00,20.0,100264,200878,셀렉프로 뚝배기 전기밥솥(멀티쿠커),0,79000,128382000,0,2019,12,28,9,0,0


## 가전

### 취급액

In [92]:
#'가전'
df_1_sold = df_1.sort_values(by=['취급액'], ascending=False)

In [132]:
# 상품명별, 마더코드별 중복 데이터 제거
top_100_1 = index_dataframe(df_1_sold, 100)
name_li_1 = column_set_to_list(top_100_1, '상품명')
mothercode_1 = column_set_to_list(top_100_1, '마더코드')

In [134]:
name_li_1

['(1등급)221L_딤채김치냉장고',
 '무이자 LG 휘센 씽큐 에어컨 2IN1형 FQ17V9KWA2',
 '무이자 LG전자 매직스페이스 냉장고',
 'LG 울트라HD TV 65UM7900BNA',
 '일시불 LG전자 매직스페이스 냉장고',
 'LG 울트라HD TV 75UM7900BNA',
 '(1등급)467L_딤채김치냉장고',
 '무이자 삼성 무풍슬림 홈멀티  에어컨  AF16R5773WSR',
 '일시불 LG 통돌이 세탁기',
 '일시불 LG 울트라HD TV 65UK6800HNC',
 '(1등급)221L_딤채김치냉장고(LDL22DHWJJ)',
 '무이자 LG 울트라HD TV 65UM7900BNA',
 '무이자 삼성 노트북 9 메탈 고급형 NT900X5J-K28',
 'LG전자 통돌이 세탁기 TR14WK1(화이트)',
 'LG 울트라HD TV 55UM7900BNA',
 '221L_딤채김치냉장고']

In [135]:
mothercode_1

[100577, 100148, 100149, 100150, 100155, 100573, 100091]

### 주문량

In [93]:
df_1 = add_column(df_1)
df_1 = df_1.sort_values(by=['주문량'], ascending=False)
df_1

C:\Users\CUCHO\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,성별 포함,년,월,일자,시,분,초,주문량
27169,2019-09-14 12:40:00,20.0,100572,201772,일시불 삼성 파워모션 청소기 VC44M3150LB,1,164000,55720000,0,2019,9,14,12,40,0,339
37768,2019-12-25 22:00:00,20.0,100150,200427,LG전자 통돌이 세탁기 TR14WK1(화이트),1,489000,144212000,0,2019,12,25,22,0,0,294
34084,2019-11-19 21:00:00,20.0,100150,200427,LG전자 통돌이 세탁기 TR14WK1(화이트),1,489000,137987000,0,2019,11,19,21,0,0,282
36118,2019-12-09 21:00:00,20.0,100150,200427,LG전자 통돌이 세탁기 TR14WK1(화이트),1,489000,132271000,0,2019,12,9,21,0,0,270
27170,2019-09-14 12:40:00,20.0,100572,201720,무이자 삼성 파워모션 청소기 VC44M3150LB,1,169000,44548000,0,2019,9,14,12,40,0,263
38074,2019-12-29 12:00:00,20.0,100150,200427,LG전자 통돌이 세탁기 TR14WK1(화이트),1,489000,125019000,0,2019,12,29,12,0,0,255
35568,2019-12-04 22:00:00,20.0,100150,200427,LG전자 통돌이 세탁기 TR14WK1(화이트),1,489000,125168000,0,2019,12,4,22,0,0,255
24735,2019-08-20 19:40:00,20.0,100572,201772,일시불 삼성 파워모션 청소기 VC44M3150LB,1,164000,39221000,0,2019,8,20,19,40,0,239
37559,2019-12-23 23:00:00,20.0,100150,200427,LG전자 통돌이 세탁기 TR14WK1(화이트),1,489000,111291000,0,2019,12,23,23,0,0,227
34083,2019-11-19 20:40:00,20.0,100150,200427,LG전자 통돌이 세탁기 TR14WK1(화이트),1,489000,110697000,0,2019,11,19,20,40,0,226


In [136]:
# 상품명별, 마더코드별 중복 데이터 제거
top_100_01 = index_dataframe(df_1, 100)
name_li_01 = column_set_to_list(top_100_01, '상품명')
mothercode_01 = column_set_to_list(top_100_01, '마더코드')

In [137]:
name_li_01

['(1등급)221L_딤채김치냉장고',
 '무이자 LG 통돌이 세탁기',
 'LG 울트라HD TV 65UM7900BNA',
 '일시불 LG전자 매직스페이스 냉장고',
 '일시불 LG 통돌이 세탁기',
 'LG전자 통돌이 세탁기 TR14WK1(화이트)',
 '무이자 LG전자 로보킹 터보 R46RIM',
 '일시불 삼성 파워모션 청소기 VC44M3150LB',
 '무이자 삼성 파워모션 청소기 VC44M3150LB']

In [138]:
mothercode_01

[100155, 100156, 100148, 100150, 100091, 100572]

### 두 기준 공통

In [241]:
popular_name_1 = list(set(name_li_1) & set(name_li_01))

In [201]:
set(mothercode_01) & set(mothercode_1)

{100091, 100148, 100150, 100155}

## 데이터프레임에서 판매 단가 비교
> 위에서 구한 상품군별 인기 상품을 데이터프레임 내에서 탐색하고, 판매단가를 비교하여 최적 상품을 찾아본다.

In [242]:
filter_df1 = filter_dataframe_for_list(load_df, '상품명', popular_name_1)
filter_df1.head(10)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,성별 포함,년,월,일자,시,분,초
32425,2019-11-03 09:20:00,30.0,100091,200248,(1등급)221L_딤채김치냉장고,1,899000,5028000,0,2019,11,3,9,20,0
32427,2019-11-03 09:50:00,30.0,100091,200248,(1등급)221L_딤채김치냉장고,1,899000,15134000,0,2019,11,3,9,50,0
32919,2019-11-07 22:20:00,30.0,100091,200248,(1등급)221L_딤채김치냉장고,1,899000,2613000,0,2019,11,7,22,20,0
32921,2019-11-07 22:50:00,30.0,100091,200248,(1등급)221L_딤채김치냉장고,1,899000,32642000,0,2019,11,7,22,50,0
33087,2019-11-09 13:20:00,20.0,100091,200248,(1등급)221L_딤채김치냉장고,1,899000,7445000,0,2019,11,9,13,20,0
33088,2019-11-09 13:40:00,20.0,100091,200248,(1등급)221L_딤채김치냉장고,1,899000,65071000,0,2019,11,9,13,40,0
33089,2019-11-09 14:00:00,20.0,100091,200248,(1등급)221L_딤채김치냉장고,1,899000,84844000,0,2019,11,9,14,0,0
33111,2019-11-09 21:20:00,30.0,100091,200248,(1등급)221L_딤채김치냉장고,1,899000,10151000,0,2019,11,9,21,20,0
33113,2019-11-09 21:50:00,30.0,100091,200248,(1등급)221L_딤채김치냉장고,1,899000,32636000,0,2019,11,9,21,50,0
33181,2019-11-10 17:20:00,30.0,100091,200248,(1등급)221L_딤채김치냉장고,1,899000,12717000,0,2019,11,10,17,20,0


In [280]:
# 상품명 기준으로 데이터 분리하기
mod=sys.modules[__name__]

for i in range(len(popular_name_1)):
    setattr(mod, 'refilter_df1_{}'.format(i), load_df[load_df['상품명']== popular_name_1[i]])

print('데이터 분리 완료')

데이터 분리 완료


In [282]:
# 상품별 데이터
refilter_df1_0.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,성별 포함,년,월,일자,시,분,초
32425,2019-11-03 09:20:00,30.0,100091,200248,(1등급)221L_딤채김치냉장고,1,899000,5028000,0,2019,11,3,9,20,0
32427,2019-11-03 09:50:00,30.0,100091,200248,(1등급)221L_딤채김치냉장고,1,899000,15134000,0,2019,11,3,9,50,0
32919,2019-11-07 22:20:00,30.0,100091,200248,(1등급)221L_딤채김치냉장고,1,899000,2613000,0,2019,11,7,22,20,0
32921,2019-11-07 22:50:00,30.0,100091,200248,(1등급)221L_딤채김치냉장고,1,899000,32642000,0,2019,11,7,22,50,0
33087,2019-11-09 13:20:00,20.0,100091,200248,(1등급)221L_딤채김치냉장고,1,899000,7445000,0,2019,11,9,13,20,0


## 의류

### 취급액

In [94]:
#'의류'
df_2_sold = df_2.sort_values(by=['취급액'], ascending=False)

In [139]:
# 상품명별, 마더코드별 중복 데이터 제거
top_100_2 = index_dataframe(df_2_sold, 100)
name_li_2 = column_set_to_list(top_100_2, '상품명')
mothercode_2 = column_set_to_list(top_100_2, '마더코드')

In [140]:
name_li_2

['CERINI by PAT 남성 스프링 릴렉스 팬츠 3종',
 '보코 폭스퍼구스다운 롱패딩',
 '코몽트 남성 프린트티셔츠8종',
 '[K-SWISS] 19 F/W 남성 패딩 트랙수트 2종',
 'NNF SS트레이닝 세트',
 '크리스티나앤코 이지웨어세트',
 '헤스티지 그레이스 양가죽 코트',
 'CERINI by PAT 남성 어반 카라 티셔츠 4종(8월)',
 'CERINI by PAT 남성 니트 스웨터 4종',
 '코몽트 남성 이너티셔츠 7종',
 '마르엘라로사티 나파 휘메일 풀스킨 밍크하프코트 1종',
 'CERINI by PAT 남성 소프트 기모 릴렉스팬츠',
 '헤스티지 엘레나 라쿤양가죽 콤비 롱코트',
 '코몽트 남성 메쉬티셔츠 7종',
 '스튜디오럭스 후드롱구스다운코트',
 '보코 풍기인견 블라우스',
 'NNF 라운지웨어세트',
 '라라쎄 쉬폰 롤업재킷 2종 세트',
 '마르엘라로사티 휘메일 풀스킨 하이넥 밍크코트 1종',
 '[헤스티지] 레나 폭스퍼후드 덕다운롱코트',
 '대동모피 19 F/W 폭스 베스트',
 '대동모피 패션밍크 후드 롱코트 (19FW)',
 '젠트웰 스트레치 통풍 메쉬 자켓 1종',
 '마르엘라로사티 에코무스탕1종',
 '마리노블 밍크 롱코트',
 '코몽트 덤블 플리스 자켓 2종',
 'CERINI by PAT 남성 에어 카라티셔츠 4종',
 'CERINI by PAT 남성 풀패키지 기모니트 3종',
 '마르엘라로사티 휘메일 풀스킨 밍크 후드코트']

In [141]:
mothercode_2

[100352,
 100356,
 100248,
 100269,
 100271,
 100272,
 100416,
 100418,
 100808,
 100809,
 100816,
 100049,
 100822,
 100062,
 100063,
 100452,
 100715,
 100462,
 100340,
 100479]

### 주문량

In [95]:
df_2 = add_column(df_2)
df_2 = df_2.sort_values(by=['주문량'], ascending=False)
df_2

C:\Users\CUCHO\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,성별 포함,년,월,일자,시,분,초,주문량
35785,2019-12-07 01:40:00,17.166667,100418,201322,코몽트 덤블 플리스 자켓 2종,2,39900,322009000,0,2019,12,7,1,40,0,8070
37357,2019-12-21 17:40:00,20.000000,100418,201322,코몽트 덤블 플리스 자켓 2종,2,39900,223492000,0,2019,12,21,17,40,0,5601
36268,2019-12-11 00:00:00,20.000000,100418,201322,코몽트 덤블 플리스 자켓 2종,2,39900,187419000,0,2019,12,11,0,0,0,4697
35797,2019-12-07 09:00:00,20.000000,100418,201322,코몽트 덤블 플리스 자켓 2종,2,39900,128955000,0,2019,12,7,9,0,0,3231
14745,2019-05-19 09:20:00,20.000000,100416,201317,코몽트 남성 이너티셔츠 7종,2,39900,117121000,1,2019,5,19,9,20,0,2935
22802,2019-08-03 08:40:00,20.000000,100416,201320,코몽트 남성 프린트티셔츠8종,2,39900,107462000,1,2019,8,3,8,40,0,2693
23587,2019-08-10 13:20:00,20.000000,100822,202422,보코 풍기인견 블라우스,2,39000,102784000,0,2019,8,10,13,20,0,2635
35796,2019-12-07 08:40:00,20.000000,100418,201322,코몽트 덤블 플리스 자켓 2종,2,39900,104910000,0,2019,12,7,8,40,0,2629
14746,2019-05-19 09:40:00,20.000000,100416,201317,코몽트 남성 이너티셔츠 7종,2,39900,101624000,1,2019,5,19,9,40,0,2546
16124,2019-06-01 08:40:00,20.000000,100416,201317,코몽트 남성 이너티셔츠 7종,2,39900,96380000,1,2019,6,1,8,40,0,2415


In [142]:
# 상품명별, 마더코드별 중복 데이터 제거
top_100_02 = index_dataframe(df_2, 100)
name_li_02 = column_set_to_list(top_100_02, '상품명')
mothercode_02 = column_set_to_list(top_100_02, '마더코드')

In [143]:
name_li_02

['스텔라테일러 컴포트 스트레치 팬츠 3종',
 '코몽트 남성 프린트티셔츠8종',
 '[K-SWISS] 19 F/W 남성 패딩 트랙수트 2종',
 'NNF SS트레이닝 세트',
 '크리스티나앤코 썸머플리츠세트(8월)',
 '헤비추얼 여성 트렌치 맥코트',
 '마르엘라로사티 롱블라우스3종',
 '아리스토우 남성기모티셔츠5종',
 'NNF 소프트 베스트 앙상블',
 '마담팰리스 웜기모 롱베스트 3종세트',
 'CERINI by PAT 남성 어반 카라 티셔츠 4종(8월)',
 '코몽트 남성 이너티셔츠 7종',
 '로이몬스터 남녀 벤치후드롱패딩',
 '크리스티나앤코 기모티셔츠세트',
 '코몽트 남성 메쉬티셔츠 7종',
 '크리스티나앤코 가디건세트',
 '스튜디오럭스 후드롱구스다운코트',
 '보코 풍기인견 블라우스',
 'NNF 와이드데님팬츠 2종',
 'NNF 라운지웨어세트',
 '라라쎄 쉬폰 롤업재킷 2종 세트',
 'NNF 쉬폰 썸머 앙상블세트',
 'NNF SS트레이닝 세트(8월)',
 '젠트웰 스트레치 통풍 메쉬 자켓 1종',
 '보코 에스닉 앙상블',
 '마르엘라로사티 에코무스탕1종',
 '코몽트 덤블 플리스 자켓 2종',
 'NNF 더블페이스 쿠션재킷+베스트']

In [144]:
mothercode_02

[100353,
 100481,
 100356,
 100248,
 100761,
 100416,
 100418,
 100809,
 100818,
 100822,
 100569,
 100452,
 100454,
 100455,
 100456,
 100458,
 100715,
 100460,
 100340,
 100342,
 100090,
 100479]

### 두 기준 공통

In [243]:
# 주문량과 취급액 top 100의 교집합 추출
popular_name_2 = list(set(name_li_02) & set(name_li_02))
popular_name_2

['스텔라테일러 컴포트 스트레치 팬츠 3종',
 '코몽트 남성 프린트티셔츠8종',
 '[K-SWISS] 19 F/W 남성 패딩 트랙수트 2종',
 'NNF SS트레이닝 세트',
 '크리스티나앤코 썸머플리츠세트(8월)',
 '헤비추얼 여성 트렌치 맥코트',
 '마르엘라로사티 롱블라우스3종',
 '아리스토우 남성기모티셔츠5종',
 'NNF 소프트 베스트 앙상블',
 '마담팰리스 웜기모 롱베스트 3종세트',
 'CERINI by PAT 남성 어반 카라 티셔츠 4종(8월)',
 '코몽트 남성 이너티셔츠 7종',
 '로이몬스터 남녀 벤치후드롱패딩',
 '크리스티나앤코 기모티셔츠세트',
 '코몽트 남성 메쉬티셔츠 7종',
 '크리스티나앤코 가디건세트',
 '스튜디오럭스 후드롱구스다운코트',
 '보코 풍기인견 블라우스',
 'NNF 와이드데님팬츠 2종',
 'NNF 라운지웨어세트',
 '라라쎄 쉬폰 롤업재킷 2종 세트',
 'NNF 쉬폰 썸머 앙상블세트',
 'NNF SS트레이닝 세트(8월)',
 '젠트웰 스트레치 통풍 메쉬 자켓 1종',
 '보코 에스닉 앙상블',
 '마르엘라로사티 에코무스탕1종',
 '코몽트 덤블 플리스 자켓 2종',
 'NNF 더블페이스 쿠션재킷+베스트']

In [202]:
# 마더코드
set(mothercode_02) & set(mothercode_2)

{100248,
 100340,
 100356,
 100416,
 100418,
 100452,
 100479,
 100715,
 100809,
 100822}

## 데이터프레임에서 판매 단가 비교
> 위에서 구한 상품군별 인기 상품을 데이터프레임 내에서 탐색하고, 판매단가를 비교하여 최적 상품을 찾아본다.

In [295]:
filter_df2 = filter_dataframe_for_list(load_df, '상품명', popular_name_2)
filter_df2.sort_values(by='취급액', ascending=False)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,성별 포함,년,월,일자,시,분,초
35785,2019-12-07 01:40:00,17.166667,100418,201322,코몽트 덤블 플리스 자켓 2종,2,39900,322009000,0,2019,12,7,1,40,0
37357,2019-12-21 17:40:00,20.000000,100418,201322,코몽트 덤블 플리스 자켓 2종,2,39900,223492000,0,2019,12,21,17,40,0
36268,2019-12-11 00:00:00,20.000000,100418,201322,코몽트 덤블 플리스 자켓 2종,2,39900,187419000,0,2019,12,11,0,0,0
32806,2019-11-07 00:00:00,20.000000,100340,201057,스튜디오럭스 후드롱구스다운코트,2,59000,135711000,0,2019,11,7,0,0,0
38262,2019-12-31 19:00:00,20.000000,100356,201134,마르엘라로사티 에코무스탕1종,2,79000,129905000,0,2019,12,31,19,0,0
35797,2019-12-07 09:00:00,20.000000,100418,201322,코몽트 덤블 플리스 자켓 2종,2,39900,128955000,0,2019,12,7,9,0,0
14745,2019-05-19 09:20:00,20.000000,100416,201317,코몽트 남성 이너티셔츠 7종,2,39900,117121000,1,2019,5,19,9,20,0
22802,2019-08-03 08:40:00,20.000000,100416,201320,코몽트 남성 프린트티셔츠8종,2,39900,107462000,1,2019,8,3,8,40,0
35796,2019-12-07 08:40:00,20.000000,100418,201322,코몽트 덤블 플리스 자켓 2종,2,39900,104910000,0,2019,12,7,8,40,0
23587,2019-08-10 13:20:00,20.000000,100822,202422,보코 풍기인견 블라우스,2,39000,102784000,0,2019,8,10,13,20,0


In [286]:
# 상품명 기준으로 데이터 분리하기
mod=sys.modules[__name__]

for i in range(len(popular_name_2)):
    setattr(mod, 'refilter_df2_{}'.format(i), load_df[load_df['상품명']== popular_name_2[i]])

print('데이터 분리 완료')

데이터 분리 완료


In [302]:
# 데이터 확인
refilter_df2_3.sort_values(by='취급액', ascending=False).head(1)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,성별 포함,년,월,일자,시,분,초
12351,2019-04-29 20:40:00,20.0,100452,201394,NNF SS트레이닝 세트,2,39000,76384000,0,2019,4,29,20,40,0


## 속옷

### 취급액

In [96]:
#'속옷'
df_3_sold = df_3.sort_values(by=['취급액'], ascending=False)

In [146]:
# 상품명별, 마더코드별 중복 데이터 제거
top_100_3 = index_dataframe(df_3_sold, 100)
name_li_3 = column_set_to_list(top_100_3, '상품명')
mothercode_3 = column_set_to_list(top_100_3, '마더코드')

In [147]:
name_li_3

['푸마 언더셔츠 7종 패키지',
 '쿠미투니카 코르셋 하이웨스트 팬티세트',
 '벨레즈온 브라캡 심리스 원피스 패키지',
 '실크트리 풍기인견 데일리 쿨 이지탑 패키지',
 '오모떼 프런트후크 프레쉬 데오브라 시즌3',
 '쿠미투니카 롱 하이웨스트 팬티세트',
 '로베르타 디 까메리노 트리플커버 지퍼쉐이핑 란쥬',
 '푸마 DRY-CELL 와이드 풀컵 브라팬티 7세트(14종)',
 '몬테밀라노 크루즈 이지웨어 패키지',
 '오모떼 데오 쿨 란쥬쉐이퍼 8종',
 '쿠미투니카 웜 레이시 란쥬패키지',
 '푸마 에어핏 서포트 브라세트 15종+4종',
 '오모떼 미라클쉐이핑 브라팬티 시즌3',
 '아키 라이크라 노와이어 브라 컬렉션(뉴아키 23차)',
 '란체티 남성 융기모 히팅 동내의 패키지',
 '리복 델타 피트니스 드로즈 12종',
 '오모떼 미라클쉐이핑 브라팬티 시즌2',
 '히트융 극세사 퍼융기모 남성 동내의 패키지',
 '푸마 DRY-DELL 기능성 티셔츠형 브라탑 패키지',
 '레이프릴 오가닉 코튼 보정팬티 12종',
 '오모떼 프런트후크 프레쉬 데오브라 시즌4',
 '히트융 극세사 퍼융기모 여성 동내의 패키지',
 '오모떼 쉐이핑 브라팬티 10종(7월)',
 '헤드 프리무빙 레깅스 4종 패키지',
 '[라쉬반］FC바로셀로나 에디션 드로즈SET',
 '푸마 드라이셀 퓨징 브라세트 패키지',
 '남영비비안 소노르 베르사유 노와이어 컬렉션 브라',
 '아키 풀커버 라이크라 레이스 패키지(106차)(랜덤2세트추가)',
 '오모떼 프런트후크 데오브라 시즌2',
 '푸마 스트레치 심리스 드로즈 11종']

In [148]:
mothercode_3

[100609,
 100098,
 100610,
 100225,
 100620,
 100275,
 100405,
 100283,
 100288,
 100305,
 100306,
 100695,
 100444,
 100445,
 100195,
 100717,
 100079,
 100080,
 100599,
 100348,
 100607]

### 주문량

In [97]:
df_3 = add_column(df_3)
df_3 = df_3.sort_values(by=['주문량'], ascending=False)
df_3

C:\Users\CUCHO\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,성별 포함,년,월,일자,시,분,초,주문량
24159,2019-08-15 11:40:00,20.00,100348,201089,벨레즈온 브라캡 심리스 원피스 패키지,3,29900,95487000,2,2019,8,15,11,40,0,3193
24158,2019-08-15 11:20:00,20.00,100348,201089,벨레즈온 브라캡 심리스 원피스 패키지,3,29900,89884000,2,2019,8,15,11,20,0,3006
34574,2019-11-25 01:00:00,20.00,100195,200655,레이프릴 오가닉 코튼 보정팬티 12종,3,39900,108559000,2,2019,11,25,1,0,0,2720
35805,2019-12-07 11:00:00,20.00,100288,200921,히트융 극세사 퍼융기모 남성 동내의 패키지,3,69900,153084000,1,2019,12,7,11,0,0,2190
23060,2019-08-05 11:40:00,20.00,100777,202274,뷰티플렉스 풍기인견 원피스 2종(8월),3,29900,64622000,2,2019,8,5,11,40,0,2161
23059,2019-08-05 11:20:00,20.00,100777,202274,뷰티플렉스 풍기인견 원피스 2종(8월),3,29900,61177000,2,2019,8,5,11,20,0,2046
37729,2019-12-25 13:40:00,20.00,100195,200655,레이프릴 오가닉 코튼 보정팬티 12종,3,39900,80335000,2,2019,12,25,13,40,0,2013
22619,2019-08-01 13:40:00,20.00,100777,202274,뷰티플렉스 풍기인견 원피스 2종(8월),3,29900,59905000,2,2019,8,1,13,40,0,2003
11818,2019-04-24 21:40:00,20.00,100283,200912,실크트리 풍기인견 데일리 쿨 이지탑 패키지,3,44000,85445000,0,2019,4,24,21,40,0,1941
37730,2019-12-25 14:00:00,20.00,100195,200655,레이프릴 오가닉 코튼 보정팬티 12종,3,39900,74243000,2,2019,12,25,14,0,0,1860


In [149]:
# 상품명별, 마더코드별 중복 데이터 제거
top_100_03 = index_dataframe(df_3, 100)
name_li_03 = column_set_to_list(top_100_03, '상품명')
mothercode_03 = column_set_to_list(top_100_03, '마더코드')

In [150]:
name_li_03

['쿠미투니카 코르셋 하이웨스트 팬티세트',
 '벨레즈온 브라캡 심리스 원피스 패키지',
 '오렐리안 히트모 스킨컬렉션 기모런닝 패키지',
 '실크트리 풍기인견 데일리 쿨 이지탑 패키지',
 '코몽트 기모 웜웨어 4종',
 '쿠미투니카 롱 하이웨스트 팬티세트',
 '로베르타 디 까메리노 트리플커버 지퍼쉐이핑 란쥬',
 '몬테밀라노 크루즈 이지웨어 패키지',
 '쿠미투니카 웜 레이시 란쥬패키지',
 '아키 라이크라 노와이어 브라 컬렉션(뉴아키 23차)',
 '에버라스트 드로즈 10+4종',
 '란체티 남성 융기모 히팅 동내의 패키지',
 '히트융 극세사 퍼융기모 남성 동내의 패키지',
 '푸마 DRY-DELL 기능성 티셔츠형 브라탑 패키지',
 '뷰티플렉스 풍기인견 원피스 2종(8월)',
 '쿠미투니카 라이크라 보정 레이스멀티팬티',
 '레이프릴 오가닉 코튼 보정팬티 12종',
 '히트융 극세사 퍼융기모 여성 동내의 패키지',
 '오모떼 쉐이핑 브라팬티 10종(7월)',
 '레이프릴 퍼펙트 핏 란쥬패키지',
 '헤드 파이로라이트 드로즈 10종 패키지(7월)',
 '아키 풀커버 라이크라 레이스 패키지(106차)(랜덤2세트추가)',
 '푸마 드라이셀 브라탑 4종세트(런닝형)(8월)',
 '오모떼 프런트후크 데오브라 시즌2']

In [151]:
mothercode_03

[100609,
 100612,
 100618,
 100632,
 100777,
 100779,
 100275,
 100405,
 100281,
 100283,
 100414,
 100288,
 100305,
 100444,
 100445,
 100192,
 100195,
 100717,
 100079,
 100080,
 100348]

### 두 가지 기준 공통

In [247]:
# 상품명
popular_name_3 = list(set(name_li_03) & set(name_li_3))

In [204]:
# 마더코드
set(mothercode_03) & set(mothercode_3)

{100079,
 100080,
 100195,
 100275,
 100283,
 100288,
 100305,
 100348,
 100405,
 100444,
 100445,
 100609,
 100717}

## 데이터프레임에서 판매 단가 비교
> 위에서 구한 상품군별 인기 상품을 데이터프레임 내에서 탐색하고, 판매단가를 비교하여 최적 상품을 찾아본다.

In [248]:
filter_df3 = filter_dataframe_for_list(load_df, '상품명', popular_name_3)
filter_df3.head(10)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,성별 포함,년,월,일자,시,분,초
29233,2019-10-03 01:00:00,20.000000,100444,201367,쿠미투니카 롱 하이웨스트 팬티세트,3,59900,16823000,0,2019,10,3,1,0,0
29234,2019-10-03 01:20:00,20.000000,100444,201367,쿠미투니카 롱 하이웨스트 팬티세트,3,59900,35356000,0,2019,10,3,1,20,0
29235,2019-10-03 01:40:00,16.683333,100444,201367,쿠미투니카 롱 하이웨스트 팬티세트,3,59900,38910000,0,2019,10,3,1,40,0
29520,2019-10-05 15:00:00,20.000000,100444,201367,쿠미투니카 롱 하이웨스트 팬티세트,3,59900,32858000,0,2019,10,5,15,0,0
29521,2019-10-05 15:20:00,20.000000,100444,201367,쿠미투니카 롱 하이웨스트 팬티세트,3,59900,77838000,0,2019,10,5,15,20,0
29522,2019-10-05 15:40:00,20.000000,100444,201367,쿠미투니카 롱 하이웨스트 팬티세트,3,59900,88913000,0,2019,10,5,15,40,0
29697,2019-10-07 06:00:00,20.000000,100444,201367,쿠미투니카 롱 하이웨스트 팬티세트,3,59900,15023000,0,2019,10,7,6,0,0
29698,2019-10-07 06:20:00,20.000000,100444,201367,쿠미투니카 롱 하이웨스트 팬티세트,3,59900,26139000,0,2019,10,7,6,20,0
29699,2019-10-07 06:40:00,20.000000,100444,201367,쿠미투니카 롱 하이웨스트 팬티세트,3,59900,43757000,0,2019,10,7,6,40,0
29814,2019-10-08 09:00:00,20.000000,100444,201367,쿠미투니카 롱 하이웨스트 팬티세트,3,59900,24528000,0,2019,10,8,9,0,0


## 잡화

### 취급액

In [98]:
#'잡화'
df_4_sold = df_4.sort_values(by=['취급액'], ascending=False)

In [152]:
# 상품명별, 마더코드별 중복 데이터 제거
top_100_4 = index_dataframe(df_4_sold, 100)
name_li_4 = column_set_to_list(top_100_4, '상품명')
mothercode_4 = column_set_to_list(top_100_4, '마더코드')

In [153]:
name_li_4

['시스마르스 플렉시블 웨지 펌프스',
 '아가타 소가죽 데이쿠션로퍼',
 '구찌 인터로킹 GG 탑핸들 체인 숄더 스몰',
 '썸덱스 클레오 여행가방 9종세트',
 '도스문도스 카이만 엠보 소가죽 핸드백 1종(역시즌)',
 '구찌 더블G 오버 선글라스',
 '일시불[가이거] 제니스시계 주얼리세트',
 '칼리베이직 여성백 3종세트',
 '일시불 썸덱스 트로이카 여행가방 10종세트',
 '아가타 골든 마스터 2종 (펌프스1종+플랫슈즈1종)',
 '생쥴랑 18K코코 천연다이아 목걸이팔찌 세트',
 '[RYN] 린 여성 다이얼락 히트 웜 퍼 방한화 2종',
 '아가타 썸머 샌들세트',
 '[루이띠에] 18k 그레이스 골드 주얼리 세트',
 'AAD 도트펀칭 컴포트 슬립온',
 '[루이띠에] 18K 베르니아 주얼리 세트',
 '[RYN] 린 남성 다이얼락 히트 웜 퍼 방한화 2종',
 '오델로 겨울모자 3종 (2차)',
 '2019 S/S 엘리자베스아덴 선글라스 2종',
 '생줄랑 14K 글로리아 목걸이 컬렉션 8종',
 '스프리스 여성 코트 스니커즈',
 '생쥴랑 18K 베니스 주얼리 컬렉션 11종',
 'AAD 소가죽 투웨이 샌들',
 '2019 S/S 기라로쉬 올인원 선글라스',
 '도스문도스 카이만 엠보 소가죽 핸드백 1종',
 '칼리베이직 2019 패브릭 백 4종 (시즌 3)',
 '오델로 여성 겨울모자 3종',
 '아가타 윈터 에디션(부츠1종+소프트퍼슈즈1종)']

In [154]:
mothercode_4

[100742,
 100751,
 100753,
 100754,
 100755,
 100757,
 100390,
 100786,
 100019,
 100021,
 100032,
 100161,
 100033,
 100036,
 100293,
 100038,
 100185,
 100069,
 100070,
 100197,
 100218]

### 주문량

In [99]:
df_4 = add_column(df_4)
df_4 = df_4.sort_values(by=['주문량'], ascending=False)
df_4

C:\Users\CUCHO\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,성별 포함,년,월,일자,시,분,초,주문량
29291,2019-10-03 14:40:00,20.0,100161,200566,시스마르스 플렉시블 웨지 펌프스,4,29800,87269000,0,2019,10,3,14,40,0,2928
29290,2019-10-03 14:20:00,20.0,100161,200566,시스마르스 플렉시블 웨지 펌프스,4,29800,77555000,0,2019,10,3,14,20,0,2602
31000,2019-10-19 14:40:00,20.0,100161,200566,시스마르스 플렉시블 웨지 펌프스,4,29800,76942000,0,2019,10,19,14,40,0,2581
37965,2019-12-28 10:00:00,20.0,100197,200664,오델로 겨울모자 3종 (2차),4,29800,73774000,0,2019,12,28,10,0,0,2475
35078,2019-11-30 10:20:00,20.0,100197,200666,오델로 여성 겨울모자 3종,4,29800,73466000,2,2019,11,30,10,20,0,2465
30999,2019-10-19 14:20:00,20.0,100161,200566,시스마르스 플렉시블 웨지 펌프스,4,29800,70776000,0,2019,10,19,14,20,0,2375
37707,2019-12-25 10:00:00,20.0,100197,200664,오델로 겨울모자 3종 (2차),4,29800,68880000,0,2019,12,25,10,0,0,2311
35076,2019-11-30 10:00:00,20.0,100197,200666,오델로 여성 겨울모자 3종,4,29800,68057000,2,2019,11,30,10,0,0,2283
28071,2019-09-22 10:40:00,20.0,100753,202202,아가타 골든 마스터 2종 (펌프스1종+플랫슈즈1종),4,59900,131176000,0,2019,9,22,10,40,0,2189
5720,2019-03-01 10:40:00,20.0,100070,200191,칼리베이직 여성백 3종세트,4,39900,87125000,2,2019,3,1,10,40,0,2183


In [155]:
# 상품명별, 마더코드별 중복 데이터 제거
top_100_04 = index_dataframe(df_4, 100)
name_li_04 = column_set_to_list(top_100_04, '상품명')
mothercode_04 = column_set_to_list(top_100_04, '마더코드')

In [156]:
name_li_04

['시스마르스 플렉시블 웨지 펌프스',
 '아가타 소가죽 데이쿠션로퍼',
 '에펨 남성 소가죽 체인지 샌들',
 '시스마르스 보헤미안 소가죽 샌들+뮬 슬리퍼',
 'AAD 여성 소가죽 구름 샌들',
 '칼리베이직 여성백 3종세트',
 '아가타 골든 마스터 2종 (펌프스1종+플랫슈즈1종)',
 '오델로 여성 모자 3종',
 '[RYN] 린 여성 다이얼락 히트 웜 퍼 방한화 2종',
 'AAD 도트펀칭 컴포트 슬립온',
 '오델로 남성 겨울모자 3종',
 '[RYN] 린 남성 다이얼락 히트 웜 퍼 방한화 2종',
 '오델로 겨울모자 3종 (2차)',
 '2019 S/S 엘리자베스아덴 선글라스 2종',
 'AAD 소가죽 투웨이 샌들',
 '2019 S/S 기라로쉬 올인원 선글라스',
 '월드컵 남성 에어러닝화+아쿠아트레킹화',
 '칼리베이직 2019 패브릭 백 4종 (시즌 3)',
 '오델로 여성 겨울모자 3종']

In [157]:
mothercode_04

[100161,
 100197,
 100070,
 100390,
 100069,
 100782,
 100753,
 100754,
 100019,
 100018,
 100021,
 100185,
 100731,
 100158]

### 두 가지 기준 공통

In [249]:
# 상품명
popular_name_4 = list(set(name_li_04) & set(name_li_4))

In [206]:
# 마더코드
set(mothercode_04) & set(mothercode_4)

{100019,
 100021,
 100069,
 100070,
 100161,
 100185,
 100197,
 100390,
 100753,
 100754}

## 데이터프레임에서 판매 단가 비교
> 위에서 구한 상품군별 인기 상품을 데이터프레임 내에서 탐색하고, 판매단가를 비교하여 최적 상품을 찾아본다.

In [250]:
filter_df4 = filter_dataframe_for_list(load_df, '상품명', popular_name_4)
filter_df4.head(10)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,성별 포함,년,월,일자,시,분,초
4730,2019-02-19 07:00:00,20.0,100070,200191,칼리베이직 여성백 3종세트,4,39900,28191000,2,2019,2,19,7,0,0
4731,2019-02-19 07:20:00,20.0,100070,200191,칼리베이직 여성백 3종세트,4,39900,40621000,2,2019,2,19,7,20,0
4732,2019-02-19 07:40:00,20.0,100070,200191,칼리베이직 여성백 3종세트,4,39900,51013000,2,2019,2,19,7,40,0
5061,2019-02-22 09:00:00,20.0,100070,200191,칼리베이직 여성백 3종세트,4,39900,23910000,2,2019,2,22,9,0,0
5062,2019-02-22 09:20:00,20.0,100070,200191,칼리베이직 여성백 3종세트,4,39900,52414000,2,2019,2,22,9,20,0
5063,2019-02-22 09:40:00,20.0,100070,200191,칼리베이직 여성백 3종세트,4,39900,59754000,2,2019,2,22,9,40,0
5245,2019-02-24 11:00:00,20.0,100070,200191,칼리베이직 여성백 3종세트,4,39900,31553000,2,2019,2,24,11,0,0
5246,2019-02-24 11:20:00,20.0,100070,200191,칼리베이직 여성백 3종세트,4,39900,58337000,2,2019,2,24,11,20,0
5247,2019-02-24 11:40:00,20.0,100070,200191,칼리베이직 여성백 3종세트,4,39900,65972000,2,2019,2,24,11,40,0
5302,2019-02-24 21:00:00,20.0,100070,200191,칼리베이직 여성백 3종세트,4,39900,18871000,2,2019,2,24,21,0,0


## 농수축

### 취급액

In [100]:
#'농수축', 
df_5_sold = df_5.sort_values(by=['취급액'], ascending=False)

In [158]:
# 상품명별, 마더코드별 중복 데이터 제거
top_100_5 = index_dataframe(df_5_sold, 100)
name_li_5 = column_set_to_list(top_100_5, '상품명')
mothercode_5 = column_set_to_list(top_100_5, '마더코드')

In [159]:
name_li_5

['멋진밥상 흥양농협 햅쌀 20kg',
 '유귀열의 The귀한 소한마리찜 7팩+무료체험1팩',
 '임성근의 녹용도가니탕 풀세트',
 '유귀열의 The 귀한 갓김치3kg+열무김치3kg, 총6kg',
 '유귀열의 귀한 갓김치3kg+총각김치3kg (총6kg)',
 '피시원 국내산 절단 햇 꽃게 7팩',
 '고창 꿀 고구마 10kg',
 '궁중 손질새우 200미 + 동태포 200g',
 '천수봉명인 선재 전통 메주세트',
 '국내산 손질 통오징어 21미',
 '안동간고등어 20팩',
 '완도꼬마활전복 1.3kg',
 '자연산 손질 통오징어 21미',
 '김선영 정성한끼 LA갈비 6팩',
 '하늘내린 용대리황태10마리',
 '제주바다 7분 손질문어',
 '참바다손질낙지100미+양념장+연포탕육수',
 '피시원 국내산 햇 손질문어 7팩',
 '하림뼈없는양념닭발세트',
 '피시원 국내산 절단 햇 꽃게 2.4kg',
 'AAB의 소곱창전골 800g x 8팩',
 '국내산 손질 햇 갑오징어 9팩',
 '피시원 국내산 햇 손질문어 8팩',
 '19년 멋진밥상 흥양농협 햅쌀 20kg',
 '국내산 손질갑오징어 9팩',
 '유귀열의 The 귀한 능이 누룽지삼계탕 8팩',
 '옛날 그 쥐포 110장 (11팩)',
 '청정수산 완도활전복 中사이즈 26미',
 '우리바다 손질왕꼬막 24팩',
 '궁중 손질새우 200미 + 동태포 400g',
 '국내산 손질갑오징어 8팩',
 '현대어찬 손질갈치']

In [160]:
mothercode_5

[100492,
 100499,
 100253,
 100511,
 100010,
 100523,
 100026,
 100548,
 100167,
 100169,
 100172,
 100435,
 100699,
 100829,
 100317,
 100318,
 100832,
 100833,
 100322,
 100323,
 100708,
 100841,
 100202,
 100844,
 100849]

### 주문량

In [101]:
df_5 = add_column(df_5)
df_5 = df_5.sort_values(by=['주문량'], ascending=False)
df_5

C:\Users\CUCHO\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,성별 포함,년,월,일자,시,분,초,주문량
33179,2019-11-10 17:00:00,20.000000,100253,200864,안동간고등어 20팩,5,30900,199541000,0,2019,11,10,17,0,0,6457
37862,2019-12-27 00:00:00,20.000000,100323,201025,피시원 국내산 햇 손질문어 7팩,5,49900,270284000,0,2019,12,27,0,0,0,5416
34386,2019-11-22 19:00:00,20.000000,100317,201665,국내산 손질 햇 갑오징어 9팩,5,49900,241079000,0,2019,11,22,19,0,0,4831
37738,2019-12-25 16:40:00,20.000000,100699,202064,고창 꿀 고구마 10kg,5,30900,133435000,0,2019,12,25,16,40,0,4318
34476,2019-11-23 23:00:00,20.000000,100548,201634,완도꼬마활전복 1.3kg,5,29900,115924000,0,2019,11,23,23,0,0,3877
32372,2019-11-02 17:40:00,20.000000,100849,202513,멋진밥상 흥양농협 햅쌀 20kg,5,58900,228177000,0,2019,11,2,17,40,0,3873
26449,2019-09-07 10:20:00,20.000000,100492,201487,피시원 국내산 절단 햇 꽃게 7팩,5,49900,181886000,0,2019,9,7,10,20,0,3645
32469,2019-11-03 17:00:00,20.000000,100548,201634,완도꼬마활전복 1.3kg,5,29900,102089000,0,2019,11,3,17,0,0,3414
35025,2019-11-29 18:00:00,20.000000,100317,201004,국내산 손질 햇 갑오징어 9팩,5,49900,166635000,0,2019,11,29,18,0,0,3339
37355,2019-12-21 17:00:00,20.000000,100317,201004,국내산 손질 햇 갑오징어 9팩,5,49900,154906000,0,2019,12,21,17,0,0,3104


In [161]:
# 상품명별, 마더코드별 중복 데이터 제거
top_100_05 = index_dataframe(df_5, 100)
name_li_05 = column_set_to_list(top_100_05, '상품명')
mothercode_05 = column_set_to_list(top_100_05, '마더코드')

In [162]:
name_li_05

['멋진밥상 흥양농협 햅쌀 20kg',
 '유귀열의 The 귀한 갓김치3kg+열무김치3kg, 총6kg',
 '유귀열의 귀한 갓김치3kg+총각김치3kg (총6kg)',
 '피시원 국내산 절단 햇 꽃게 7팩',
 '고창 꿀 고구마 10kg',
 '궁중 손질새우 200미 + 동태포 200g',
 '강원도양구 간편시래기15팩',
 '국내산 손질 통오징어 21미',
 '완도꼬마활전복 1.3kg',
 '안동간고등어 20팩',
 '자연산 손질 통오징어 21미',
 '하늘내린 용대리황태10마리',
 '참바다손질낙지100미+양념장+연포탕육수',
 '영광 生 모시떡 100개!!(동부기피70개+검정깨30개)',
 '피시원 국내산 햇 손질문어 7팩',
 '하림뼈없는양념닭발세트',
 '피시원 국내산 절단 햇 꽃게 2.4kg',
 '영산포숙성홍어회7팩',
 '국내산 손질 햇 갑오징어 9팩',
 '피시원 국내산 햇 손질문어 8팩',
 '유귀열의 The 귀한 능이 누룽지삼계탕 8팩',
 '옛날 그 쥐포 110장 (11팩)',
 '청정수산 완도활전복 中사이즈 26미',
 '우리바다 손질왕꼬막 24팩',
 '국내산 손질갑오징어 8팩',
 '현대어찬 손질갈치']

In [164]:
mothercode_05

[100492,
 100499,
 100253,
 100637,
 100511,
 100523,
 100026,
 100548,
 100166,
 100167,
 100169,
 100435,
 100698,
 100699,
 100829,
 100317,
 100832,
 100322,
 100323,
 100841,
 100844,
 100849]

### 두 기준 공통

In [251]:
# 상품명
popular_name_5 = list(set(name_li_05) & set(name_li_5))

In [208]:
# 마더코드
set(mothercode_05) & set(mothercode_5)

{100026,
 100167,
 100169,
 100253,
 100317,
 100322,
 100323,
 100435,
 100492,
 100499,
 100511,
 100523,
 100548,
 100699,
 100829,
 100832,
 100841,
 100844,
 100849}

## 데이터프레임에서 판매 단가 비교
> 위에서 구한 상품군별 인기 상품을 데이터프레임 내에서 탐색하고, 판매단가를 비교하여 최적 상품을 찾아본다.

In [ ]:
filter_df5 = filter_dataframe_for_list(load_df, '상품명', popular_name_5)
filter_df5.head(10)

In [267]:
filter_df5['상품명'].value_counts()

안동간고등어 20팩                          318
국내산 손질갑오징어 8팩                       160
옛날 그 쥐포 110장 (11팩)                  116
유귀열의 귀한 갓김치3kg+총각김치3kg (총6kg)       104
멋진밥상 흥양농협 햅쌀 20kg                    75
자연산 손질 통오징어 21미                      74
하림뼈없는양념닭발세트                          69
우리바다 손질왕꼬막 24팩                       60
피시원 국내산 햇 손질문어 7팩                    60
국내산 손질 햇 갑오징어 9팩                     53
하늘내린 용대리황태10마리                       47
국내산 손질 통오징어 21미                      45
유귀열의 The 귀한 능이 누룽지삼계탕 8팩             45
피시원 국내산 햇 손질문어 8팩                    42
궁중 손질새우 200미 + 동태포 200g              30
유귀열의 The 귀한 갓김치3kg+열무김치3kg, 총6kg     27
참바다손질낙지100미+양념장+연포탕육수                27
청정수산 완도활전복 中사이즈 26미                  21
완도꼬마활전복 1.3kg                        19
피시원 국내산 절단 햇 꽃게 7팩                   15
피시원 국내산 절단 햇 꽃게 2.4kg                15
고창 꿀 고구마 10kg                         6
Name: 상품명, dtype: int64

In [274]:
# 상품명 기준으로 데이터 분리하기
mod=sys.modules[__name__]

for i in range(len(popular_name_5)):
    setattr(mod, 'refilter_df0_{}'.format(i), load_df[load_df['상품명']== popular_name_5[i]])

print('데이터 분리 완료')

데이터 분리 완료


In [277]:
refilter_df0_0.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,성별 포함,년,월,일자,시,분,초
35,2019-01-01 16:40:00,20.0,100849,202512,멋진밥상 흥양농협 햅쌀 20kg,5,61000,83893000,0,2019,1,1,16,40,0
386,2019-01-05 16:00:00,20.0,100849,202512,멋진밥상 흥양농협 햅쌀 20kg,5,61000,44297000,0,2019,1,5,16,0,0
387,2019-01-05 16:20:00,20.0,100849,202512,멋진밥상 흥양농협 햅쌀 20kg,5,61000,65082000,0,2019,1,5,16,20,0
388,2019-01-05 16:40:00,20.0,100849,202512,멋진밥상 흥양농협 햅쌀 20kg,5,61000,69522000,0,2019,1,5,16,40,0
579,2019-01-07 15:00:00,20.0,100849,202512,멋진밥상 흥양농협 햅쌀 20kg,5,61000,31814000,0,2019,1,7,15,0,0


## 생활용품

### 취급액

In [102]:
#'생활용품'
df_6_sold = df_6.sort_values(by=['취급액'], ascending=False)

In [165]:
# 상품명별, 마더코드별 중복 데이터 제거
top_100_6 = index_dataframe(df_6_sold, 100)
name_li_6 = column_set_to_list(top_100_6, '상품명')
mothercode_6 = column_set_to_list(top_100_6, '마더코드')

In [166]:
name_li_6

['일시불 2019년 타이거 충전식 무선 예초기+배터리 1개',
 '무이자  얼리젼 밀레 푸쉬풀 도어락',
 '코이모 리빙박스 8종',
 '일시불 국내제조 노비타 뉴스파 비데 무료설치',
 '사운드룩 올인원 LP턴테이블 오디오 SLT-5080PRO',
 '디비노 선글라스 세트',
 '노비타 스마트 비데',
 'LG생활건강 샤프란아우라 고농축 섬유유연제',
 '코튼데이 유기농 순면 마스크 KF94 60매',
 '킹스스파 편백 반신욕기',
 '(퀸+싱글)일월 품안애 온수매트',
 '바로바로 무선청소기',
 '파격가 노비타 스마트 비데 무료설치',
 '발렌티노루디 헤링본 리빙박스 6종',
 '파격가 니봇 무선 진공 물걸레 로봇청소기 +폴더블 헤어드라이어기',
 '일시불 선일금고 이볼브 시리즈 EV-040',
 '테팔 에어포스 라이트 무선 청소기',
 '스팀큐다리미',
 '(퀸+퀸)일월 품안애 온수매트']

In [167]:
mothercode_6

[100450,
 100226,
 100421,
 100379,
 100330,
 100176,
 100563,
 100372,
 100501,
 100763,
 100182,
 100469,
 100052,
 100635,
 100029]

### 주문량

In [103]:
df_6 = add_column(df_6)
df_6 = df_6.sort_values(by=['주문량'], ascending=False)
df_6

C:\Users\CUCHO\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,성별 포함,년,월,일자,시,분,초,주문량
37160,2019-12-19 13:00:00,20.0,100330,201036,LG생활건강 샤프란아우라 고농축 섬유유연제,6,30900,87154000,0,2019,12,19,13,0,0,2820
6300,2019-03-06 13:40:00,20.0,100450,201392,코튼데이 유기농 순면 마스크 KF94 60매,6,39900,96564000,0,2019,3,6,13,40,0,2420
36470,2019-12-13 07:00:00,20.0,100330,201036,LG생활건강 샤프란아우라 고농축 섬유유연제,6,30900,70501000,0,2019,12,13,7,0,0,2281
37159,2019-12-19 12:40:00,20.0,100330,201036,LG생활건강 샤프란아우라 고농축 섬유유연제,6,30900,68452000,0,2019,12,19,12,40,0,2215
29933,2019-10-09 12:40:00,20.0,100226,200786,발렌티노루디 헤링본 리빙박스 6종,6,40800,86038000,0,2019,10,9,12,40,0,2108
6299,2019-03-06 13:20:00,20.0,100450,201392,코튼데이 유기농 순면 마스크 KF94 60매,6,39900,72434000,0,2019,3,6,13,20,0,1815
8893,2019-03-31 13:40:00,20.0,100226,201019,코이모 리빙박스 8종,6,49800,79872000,0,2019,3,31,13,40,0,1603
36469,2019-12-13 06:40:00,20.0,100330,201036,LG생활건강 샤프란아우라 고농축 섬유유연제,6,30900,48551000,0,2019,12,13,6,40,0,1571
6620,2019-03-09 13:40:00,20.0,100450,201392,코튼데이 유기농 순면 마스크 KF94 60매,6,39900,62549000,0,2019,3,9,13,40,0,1567
16549,2019-06-04 23:40:00,20.0,100469,201436,디비노 선글라스 세트,6,44800,69851000,0,2019,6,4,23,40,0,1559


In [171]:
# 상품명별, 마더코드별 중복 데이터 제거
top_100_06 = index_dataframe(df_6, 100)
name_li_06 = column_set_to_list(top_100_06, '상품명')
mothercode_06 = column_set_to_list(top_100_06, '마더코드')

In [172]:
name_li_06

['국내제조 벨라홈 논슬립 옷걸이 세트',
 'LG생활건강 샤프란아우라 고농축 섬유유연제',
 '올바로바른자세밴드 2종세트',
 '까사마루행거 2단',
 '초특가 센스하우스 리빙박스',
 '벨라홈 논슬립 옷걸이 세트',
 '바로바로 무선청소기',
 '코이모 리빙박스 8종',
 '가)세렌셉템버 다이아 체크 리빙박스+압축팩',
 '이지스 자외선 마스크 세트',
 '스팀큐다리미',
 '디비노 선글라스 세트',
 '코튼데이 유기농 순면 마스크 KF94 60매',
 'KF94 엔웰스황사방역마스크 100매',
 '발렌티노루디 헤링본 리빙박스 6종']

In [173]:
mothercode_06

[100450,
 100226,
 100229,
 100134,
 100201,
 100330,
 100239,
 100052,
 100469,
 100470,
 100635,
 100316]

### 두 기준 공통

In [289]:
# 상품명
popular_name_6 = list(set(name_li_06) & set(name_li_6))

In [210]:
# 마더코드
set(mothercode_6) & set(mothercode_06)

{100052, 100226, 100330, 100450, 100469, 100635}

In [292]:
filter_df6 = filter_dataframe_for_list(load_df, '상품명', popular_name_6)
filter_df6.head(10)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,성별 포함,년,월,일자,시,분,초
36468,2019-12-13 06:20:00,20.0,100330,201036,LG생활건강 샤프란아우라 고농축 섬유유연제,6,30900,40565000,0,2019,12,13,6,20,0
36469,2019-12-13 06:40:00,20.0,100330,201036,LG생활건강 샤프란아우라 고농축 섬유유연제,6,30900,48551000,0,2019,12,13,6,40,0
36470,2019-12-13 07:00:00,20.0,100330,201036,LG생활건강 샤프란아우라 고농축 섬유유연제,6,30900,70501000,0,2019,12,13,7,0,0
37158,2019-12-19 12:20:00,20.0,100330,201036,LG생활건강 샤프란아우라 고농축 섬유유연제,6,30900,41036000,0,2019,12,19,12,20,0
37159,2019-12-19 12:40:00,20.0,100330,201036,LG생활건강 샤프란아우라 고농축 섬유유연제,6,30900,68452000,0,2019,12,19,12,40,0
37160,2019-12-19 13:00:00,20.0,100330,201036,LG생활건강 샤프란아우라 고농축 섬유유연제,6,30900,87154000,0,2019,12,19,13,0,0
4948,2019-02-21 09:00:00,20.0,100635,201945,바로바로 무선청소기,6,89000,22227000,0,2019,2,21,9,0,0
4949,2019-02-21 09:20:00,20.0,100635,201945,바로바로 무선청소기,6,89000,43751000,0,2019,2,21,9,20,0
4950,2019-02-21 09:40:00,20.0,100635,201945,바로바로 무선청소기,6,89000,60550000,0,2019,2,21,9,40,0
5230,2019-02-24 09:00:00,20.0,100635,201945,바로바로 무선청소기,6,89000,36366000,0,2019,2,24,9,0,0


In [294]:
filter_df6.sort_values(by='취급액', ascending=False)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,성별 포함,년,월,일자,시,분,초
6300,2019-03-06 13:40:00,20.000000,100450,201392,코튼데이 유기농 순면 마스크 KF94 60매,6,39900,96564000,0,2019,3,6,13,40,0
5232,2019-02-24 09:40:00,20.000000,100635,201945,바로바로 무선청소기,6,89000,91492000,0,2019,2,24,9,40,0
37160,2019-12-19 13:00:00,20.000000,100330,201036,LG생활건강 샤프란아우라 고농축 섬유유연제,6,30900,87154000,0,2019,12,19,13,0,0
5836,2019-03-02 11:40:00,20.000000,100635,201945,바로바로 무선청소기,6,89000,82750000,0,2019,3,2,11,40,0
8893,2019-03-31 13:40:00,20.000000,100226,201019,코이모 리빙박스 8종,6,49800,79872000,0,2019,3,31,13,40,0
28847,2019-09-29 14:40:00,20.000000,100226,201022,코이모 리빙박스 8종,6,52900,73316000,0,2019,9,29,14,40,0
6299,2019-03-06 13:20:00,20.000000,100450,201392,코튼데이 유기농 순면 마스크 KF94 60매,6,39900,72434000,0,2019,3,6,13,20,0
8767,2019-03-30 10:40:00,20.000000,100226,201019,코이모 리빙박스 8종,6,49800,71595000,0,2019,3,30,10,40,0
36470,2019-12-13 07:00:00,20.000000,100330,201036,LG생활건강 샤프란아우라 고농축 섬유유연제,6,30900,70501000,0,2019,12,13,7,0,0
16549,2019-06-04 23:40:00,20.000000,100469,201436,디비노 선글라스 세트,6,44800,69851000,0,2019,6,4,23,40,0


In [290]:
# 상품명 기준으로 데이터 분리하기
mod=sys.modules[__name__]

for i in range(len(popular_name_6)):
    setattr(mod, 'refilter_df6_{}'.format(i), load_df[load_df['상품명']== popular_name_6[i]])

print('데이터 분리 완료')

데이터 분리 완료


In [291]:
# 데이터 확인
refilter_df6_1.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,성별 포함,년,월,일자,시,분,초
4948,2019-02-21 09:00:00,20.0,100635,201945,바로바로 무선청소기,6,89000,22227000,0,2019,2,21,9,0,0
4949,2019-02-21 09:20:00,20.0,100635,201945,바로바로 무선청소기,6,89000,43751000,0,2019,2,21,9,20,0
4950,2019-02-21 09:40:00,20.0,100635,201945,바로바로 무선청소기,6,89000,60550000,0,2019,2,21,9,40,0
5230,2019-02-24 09:00:00,20.0,100635,201945,바로바로 무선청소기,6,89000,36366000,0,2019,2,24,9,0,0
5231,2019-02-24 09:20:00,20.0,100635,201945,바로바로 무선청소기,6,89000,69617000,0,2019,2,24,9,20,0


## 가구

### 취급액

In [104]:
#'가구'
df_7_sold = df_7.sort_values(by=['취급액'], ascending=False)

In [174]:
# 상품명별, 마더코드별 중복 데이터 제거
top_100_7 = index_dataframe(df_7_sold, 100)
name_li_7 = column_set_to_list(top_100_7, '상품명')
mothercode_7 = column_set_to_list(top_100_7, '마더코드')

In [175]:
name_li_7

['유캐슬 마리노 침실가구 풀세트 (매트리스포함)',
 '보루네오 루나 유로탑 멀티수납형 LED 침대 Q 퀸',
 '(일) 삼익가구 LED 제니비 서랍형 침대 SS',
 '(일)장수흙침대 퀸',
 '보루네오 루나시즌2 유로탑 멀티수납형 LED침대 킹',
 '보루네오 루나 유로탑 멀티수납형 LED 침대 SS 슈퍼싱글',
 '보루네오 루나 유로탑 멀티수납형 LED 침대 K 킹',
 '(일)삼익가구 어블러 LED 멀티 수납형 침대 슈퍼싱글',
 '(무) 한샘 하이바스 내추럴 기본형',
 '(무)장수흙침대 퀸',
 '보루네오 루나시즌2 유로탑 멀티수납형 LED침대 퀸',
 '(일)[보루네오] 피올레 천연소가죽 소파 4인용',
 '(무)[보루네오] 피올레 천연소가죽 소파 4인용',
 '보루네오 루나시즌2 유로탑 멀티수납형 LED침대 슈퍼싱글']

In [176]:
mothercode_7

[100203, 100075, 100205, 100500, 100373, 100502]

### 주문량

In [105]:
df_7 = add_column(df_7)
df_7 = df_7.sort_values(by=['주문량'], ascending=False)
df_7

C:\Users\CUCHO\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,성별 포함,년,월,일자,시,분,초,주문량
27028,2019-09-13 01:20:00,20.000000,100230,200793,뉴 벨라홈 리클라이닝 소파,7,89000,20675000,0,2019,9,13,1,20,0,232
28682,2019-09-28 01:40:00,16.850000,100230,200793,뉴 벨라홈 리클라이닝 소파,7,89000,19073000,0,2019,9,28,1,40,0,214
28681,2019-09-28 01:20:00,20.000000,100230,200793,뉴 벨라홈 리클라이닝 소파,7,89000,17520000,0,2019,9,28,1,20,0,196
27029,2019-09-13 01:40:00,16.833333,100230,200793,뉴 벨라홈 리클라이닝 소파,7,89000,17527000,0,2019,9,13,1,40,0,196
29231,2019-10-03 00:40:00,20.000000,100206,200740,보루네오 심플 광폭 서랍장 1000 5단,7,199000,30828000,0,2019,10,3,0,40,0,154
27768,2019-09-19 12:40:00,20.000000,100206,200740,보루네오 심플 광폭 서랍장 1000 5단,7,199000,29832000,0,2019,9,19,12,40,0,149
32522,2019-11-04 01:00:00,20.000000,100206,200740,보루네오 심플 광폭 서랍장 1000 5단,7,199000,23846000,0,2019,11,4,1,0,0,119
30337,2019-10-12 23:00:00,20.000000,100205,200735,보루네오 루나시즌2 유로탑 멀티수납형 LED침대 슈퍼싱글,7,548000,65618000,0,2019,10,12,23,0,0,119
3688,2019-02-08 21:40:00,20.000000,100205,200733,보루네오 루나 유로탑 멀티수납형 LED 침대 SS 슈퍼싱글,7,548000,63277000,0,2019,2,8,21,40,0,115
5112,2019-02-22 21:40:00,20.000000,100205,200733,보루네오 루나 유로탑 멀티수납형 LED 침대 SS 슈퍼싱글,7,548000,58879000,0,2019,2,22,21,40,0,107


In [177]:
# 상품명별, 마더코드별 중복 데이터 제거
top_100_07 = index_dataframe(df_7, 100)
name_li_07 = column_set_to_list(top_100_07, '상품명')
mothercode_07 = column_set_to_list(top_100_07, '마더코드')

In [178]:
name_li_07

['보루네오 심플 광폭 서랍장 1000 5단',
 '보루네오 루나 유로탑 멀티수납형 LED 침대 Q 퀸',
 '(일) 삼익가구 LED 제니비 서랍형 침대 SS',
 '보루네오 루나시즌2 유로탑 멀티수납형 LED침대 킹',
 '보루네오 루나 유로탑 멀티수납형 LED 침대 SS 슈퍼싱글',
 '보루네오 루나 유로탑 멀티수납형 LED 침대 K 킹',
 '(일)삼익가구 어블러 LED 멀티 수납형 침대 슈퍼싱글',
 '보루네오 루나시즌2 유로탑 멀티수납형 LED침대 퀸',
 '보루네오 심플 광폭 서랍장 800 5단',
 '보루네오 루나시즌2 유로탑 멀티수납형 LED침대 슈퍼싱글',
 '뉴 벨라홈 리클라이닝 소파']

In [179]:
mothercode_07

[100206, 100500, 100205, 100230]

### 두 기준 공통

In [312]:
# 상품명
popular_name_7 = list(set(name_li_07) & set(name_li_7))

In [212]:
# 마더코드
set(mothercode_07) & set(mothercode_7)

{100205, 100500}

## 데이터프레임에서 판매 단가 비교
> 위에서 구한 상품군별 인기 상품을 데이터프레임 내에서 탐색하고, 판매단가를 비교하여 최적 상품을 찾아본다.

In [313]:
filter_df7 = filter_dataframe_for_list(load_df, '상품명', popular_name_7)
filter_df7.sort_values(by='취급액', ascending=False)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,성별 포함,년,월,일자,시,분,초
3688,2019-02-08 21:40:00,20.0,100205,200733,보루네오 루나 유로탑 멀티수납형 LED 침대 SS 슈퍼싱글,7,548000,63277000,0,2019,2,8,21,40,0
31240,2019-10-21 21:40:00,20.0,100205,200739,보루네오 루나시즌2 유로탑 멀티수납형 LED침대 킹,7,648000,62465000,0,2019,10,21,21,40,0
5112,2019-02-22 21:40:00,20.0,100205,200733,보루네오 루나 유로탑 멀티수납형 LED 침대 SS 슈퍼싱글,7,548000,58879000,0,2019,2,22,21,40,0
6026,2019-03-03 21:40:00,20.0,100205,200733,보루네오 루나 유로탑 멀티수납형 LED 침대 SS 슈퍼싱글,7,548000,55067000,0,2019,3,3,21,40,0
28278,2019-09-23 21:40:00,20.0,100205,200739,보루네오 루나시즌2 유로탑 멀티수납형 LED침대 킹,7,648000,52863000,0,2019,9,23,21,40,0
25788,2019-08-31 23:00:00,20.0,100205,200739,보루네오 루나시즌2 유로탑 멀티수납형 LED침대 킹,7,648000,52265000,0,2019,8,31,23,0,0
7571,2019-03-18 20:40:00,20.0,100205,200733,보루네오 루나 유로탑 멀티수납형 LED 침대 SS 슈퍼싱글,7,548000,52108000,0,2019,3,18,20,40,0
17847,2019-06-16 22:00:00,20.0,100205,200731,보루네오 루나 유로탑 멀티수납형 LED 침대 K 킹,7,648000,50969000,0,2019,6,16,22,0,0
8834,2019-03-30 22:00:00,20.0,100205,200733,보루네오 루나 유로탑 멀티수납형 LED 침대 SS 슈퍼싱글,7,548000,50600000,0,2019,3,30,22,0,0
13069,2019-05-05 21:00:00,20.0,100205,200733,보루네오 루나 유로탑 멀티수납형 LED 침대 SS 슈퍼싱글,7,548000,50528000,0,2019,5,5,21,0,0


## 이미용

### 취급액

In [106]:
#'이미용'
df_8_sold = df_8.sort_values(by=['취급액'], ascending=False)

In [180]:
# 상품명별, 마더코드별 중복 데이터 제거
top_100_8 = index_dataframe(df_8_sold, 100)
name_li_8 = column_set_to_list(top_100_8, '상품명')
mothercode_8 = column_set_to_list(top_100_8, '마더코드')

In [181]:
name_li_8

['비버리힐스폴로클럽 남성기초세트(골드+플라)',
 '비버리힐스폴로클럽 남성기초세트(골드+콜라겐)',
 '비버리힐스폴로클럽 남성기초세트(2018FW골드)',
 '바바코코 젤네일스트립 SET',
 '비버리힐스폴로클럽 남성기초세트(플라센타아이크림+선크림)',
 '참존 탑뉴스 지이링클 기초세트',
 '네오젠 에이지큐어 원미닛 링클 리프트',
 '[VONIN]보닌 남성기초세트',
 '라라츄 틴트 아이브로우 바',
 '스포메틱스 네츄럴 선스틱']

In [182]:
mothercode_8

[100327, 100074, 100399, 100211, 100244, 100247, 100766]

### 주문량

In [107]:
df_8 = add_column(df_8)
df_8 = df_8.sort_values(by=['주문량'], ascending=False)
df_8

C:\Users\CUCHO\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,성별 포함,년,월,일자,시,분,초,주문량
37438,2019-12-22 17:00:00,20.000000,100074,201007,비버리힐스폴로클럽 남성기초세트(골드+콜라겐),8,39800,177875000,1,2019,12,22,17,0,0,4469
34668,2019-11-25 22:00:00,20.000000,100074,200199,비버리힐스폴로클럽 남성기초세트(골드+콜라겐),8,39800,161654000,1,2019,11,25,22,0,0,4061
34903,2019-11-28 15:00:00,20.000000,100766,202239,참존 탑뉴스 지이링클 기초세트,8,59000,185927000,0,2019,11,28,15,0,0,3151
37082,2019-12-18 22:00:00,20.000000,100074,201007,비버리힐스폴로클럽 남성기초세트(골드+콜라겐),8,39800,123014000,1,2019,12,18,22,0,0,3090
33192,2019-11-10 21:00:00,20.000000,100074,200199,비버리힐스폴로클럽 남성기초세트(골드+콜라겐),8,39800,120367000,1,2019,11,10,21,0,0,3024
36885,2019-12-16 21:00:00,20.000000,100327,201030,[VONIN]보닌 남성기초세트,8,54000,162828000,1,2019,12,16,21,0,0,3015
38,2019-01-01 17:40:00,20.000000,100074,200197,비버리힐스폴로클럽 남성기초세트(2018FW골드),8,39800,107103000,1,2019,1,1,17,40,0,2691
1119,2019-01-13 10:40:00,20.000000,100074,200197,비버리힐스폴로클럽 남성기초세트(2018FW골드),8,39800,104268000,1,2019,1,13,10,40,0,2619
35220,2019-12-01 19:40:00,20.000000,100074,200199,비버리힐스폴로클럽 남성기초세트(골드+콜라겐),8,39800,104111000,1,2019,12,1,19,40,0,2615
33853,2019-11-17 13:00:00,20.000000,100074,200199,비버리힐스폴로클럽 남성기초세트(골드+콜라겐),8,39800,102503000,1,2019,11,17,13,0,0,2575


In [183]:
# 상품명별, 마더코드별 중복 데이터 제거
top_100_08 = index_dataframe(df_8, 100)
name_li_08 = column_set_to_list(top_100_08, '상품명')
mothercode_08 = column_set_to_list(top_100_08, '마더코드')

In [184]:
name_li_08

['비버리힐스폴로클럽 남성기초세트(골드+플라)',
 '비버리힐스폴로클럽 남성기초세트(골드+콜라겐)',
 '비버리힐스폴로클럽 남성기초세트(2018FW골드)',
 '비버리힐스폴로클럽 남성기초세트(플라센타아이크림+선크림)',
 '참존 탑뉴스 지이링클 기초세트',
 '[VONIN]보닌 남성기초세트',
 '스포메틱스 네츄럴 선스틱']

In [185]:
mothercode_08

[100074, 100211, 100766, 100327]

### 두 기준 공통

In [253]:
# 상품명
popular_name_8 =list(set(name_li_08) & set(name_li_8))

In [214]:
# 마더코드
set(mothercode_08) & set(mothercode_8)

{100074, 100211, 100327, 100766}

## 데이터프레임에서 판매 단가 비교
> 위에서 구한 상품군별 인기 상품을 데이터프레임 내에서 탐색하고, 판매단가를 비교하여 최적 상품을 찾아본다.

In [296]:
filter_df8 = filter_dataframe_for_list(load_df, '상품명', popular_name_8)
filter_df8.sort_values(by='취급액', ascending=False)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,성별 포함,년,월,일자,시,분,초
34903,2019-11-28 15:00:00,20.000000,100766,202239,참존 탑뉴스 지이링클 기초세트,8,59000,185927000,0,2019,11,28,15,0,0
37438,2019-12-22 17:00:00,20.000000,100074,201007,비버리힐스폴로클럽 남성기초세트(골드+콜라겐),8,39800,177875000,1,2019,12,22,17,0,0
36885,2019-12-16 21:00:00,20.000000,100327,201030,[VONIN]보닌 남성기초세트,8,54000,162828000,1,2019,12,16,21,0,0
34668,2019-11-25 22:00:00,20.000000,100074,200199,비버리힐스폴로클럽 남성기초세트(골드+콜라겐),8,39800,161654000,1,2019,11,25,22,0,0
34561,2019-11-24 21:20:00,20.000000,100766,202239,참존 탑뉴스 지이링클 기초세트,8,59000,132334000,0,2019,11,24,21,20,0
32422,2019-11-03 08:40:00,20.000000,100327,201032,[VONIN]보닌 남성기초세트,8,54000,130244000,1,2019,11,3,8,40,0
37082,2019-12-18 22:00:00,20.000000,100074,201007,비버리힐스폴로클럽 남성기초세트(골드+콜라겐),8,39800,123014000,1,2019,12,18,22,0,0
33192,2019-11-10 21:00:00,20.000000,100074,200199,비버리힐스폴로클럽 남성기초세트(골드+콜라겐),8,39800,120367000,1,2019,11,10,21,0,0
37765,2019-12-25 21:00:00,20.000000,100766,202239,참존 탑뉴스 지이링클 기초세트,8,59000,111109000,0,2019,12,25,21,0,0
35866,2019-12-07 21:20:00,20.000000,100766,202239,참존 탑뉴스 지이링클 기초세트,8,59000,109113000,0,2019,12,7,21,20,0


In [283]:
# 상품명 기준으로 데이터 분리하기
mod=sys.modules[__name__]

for i in range(len(popular_name_8)):
    setattr(mod, 'refilter_df8_{}'.format(i), load_df[load_df['상품명']== popular_name_8[i]])

print('데이터 분리 완료')

데이터 분리 완료


In [285]:
# 데이터 확인
refilter_df8_0.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,성별 포함,년,월,일자,시,분,초
8305,2019-03-25 17:00:00,20.0,100074,200200,비버리힐스폴로클럽 남성기초세트(골드+플라),8,39800,19048000,1,2019,3,25,17,0,0
8306,2019-03-25 17:20:00,20.0,100074,200200,비버리힐스폴로클럽 남성기초세트(골드+플라),8,39800,34889000,1,2019,3,25,17,20,0
8307,2019-03-25 17:40:00,20.0,100074,200200,비버리힐스폴로클럽 남성기초세트(골드+플라),8,39800,41633000,1,2019,3,25,17,40,0
8409,2019-03-26 18:00:00,20.0,100074,200200,비버리힐스폴로클럽 남성기초세트(골드+플라),8,39800,26021000,1,2019,3,26,18,0,0
8410,2019-03-26 18:20:00,20.0,100074,200200,비버리힐스폴로클럽 남성기초세트(골드+플라),8,39800,40357000,1,2019,3,26,18,20,0


## 건강기능

### 취급액

In [215]:
#'건강기능'
df_9_sold = df_9.sort_values(by=['취급액'], ascending=False)

In [216]:
# 상품명별, 마더코드별 중복 데이터 제거
top_100_9 = index_dataframe(df_9_sold, 100)
name_li_9 = column_set_to_list(top_100_9, '상품명')
mothercode_9 = column_set_to_list(top_100_9, '마더코드')

In [217]:
name_li_9

['통째로 착즙한 루비 석류즙 84포',
 '리얼니파팜 해죽순',
 '일시불 종근당건강 락토핏 생유산균골드 단하루',
 '닥터 슈퍼 루테인',
 '무농약레드비트즙 90봉',
 '" 100%리얼 착즙 석류 ", 총 4박스',
 '[광동제약] 석류 100, 총 5박스',
 '정직한 농부의 석류',
 '제주농장 유기 양배추진액 120포',
 '  뉴트리원 다이어트 구미',
 '(직매입) 종근당건강 아이클리어 루테인']

In [218]:
mothercode_9

[100321,
 100315,
 100651,
 100654,
 100558,
 100622,
 100562,
 100532,
 100309,
 100055,
 100539]

### 주문량

In [219]:
df_9 = add_column(df_9)
df_9 = df_9.sort_values(by=['주문량'], ascending=False)
df_9

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,성별 포함,년,월,일자,시,분,초,주문량
37257,2019-12-20 14:00:00,20.000000,100558,201655,정직한 농부의 석류,9,59900,216065000,0,2019,12,20,14,0,0,3607
35301,2019-12-02 13:00:00,20.000000,100562,201659,무농약레드비트즙 90봉,9,39900,56787000,0,2019,12,2,13,0,0,1423
17838,2019-06-16 21:00:00,20.000000,100055,200139,뉴트리원 다이어트 구미,9,59000,74027000,0,2019,6,16,21,0,0,1254
37321,2019-12-21 09:00:00,20.000000,100558,201655,정직한 농부의 석류,9,59900,73332000,0,2019,12,21,9,0,0,1224
7419,2019-03-17 07:40:00,20.000000,100309,200992,닥터 슈퍼 루테인,9,59900,72875000,0,2019,3,17,7,40,0,1216
17302,2019-06-11 23:40:00,20.000000,100055,200139,뉴트리원 다이어트 구미,9,59000,69082000,0,2019,6,11,23,40,0,1170
28212,2019-09-23 08:40:00,20.000000,100562,201659,무농약레드비트즙 90봉,9,39900,45213000,0,2019,9,23,8,40,0,1133
33682,2019-11-15 20:00:00,20.000000,100654,201980,일시불 종근당건강 락토핏 생유산균골드 단하루,9,75050,85055000,0,2019,11,15,20,0,0,1133
37991,2019-12-28 14:00:00,20.000000,100558,201655,정직한 농부의 석류,9,59900,67656000,0,2019,12,28,14,0,0,1129
18772,2019-06-26 21:40:00,20.000000,100055,200139,뉴트리원 다이어트 구미,9,59000,65815000,0,2019,6,26,21,40,0,1115


In [220]:
# 상품명별, 마더코드별 중복 데이터 제거
top_100_09 = index_dataframe(df_9, 100)
name_li_09 = column_set_to_list(top_100_09, '상품명')
mothercode_09 = column_set_to_list(top_100_09, '마더코드')

In [221]:
name_li_09

['통째로 착즙한 루비 석류즙 84포',
 '일시불 종근당건강 락토핏 생유산균골드 단하루',
 '닥터 슈퍼 루테인',
 '무농약레드비트즙 90봉',
 '[광동제약] 석류 100, 총 5박스',
 '경남제약 레모나 S산 350포',
 '정직한 농부의 석류',
 '제주농장 유기 양배추진액 120포',
 '  뉴트리원 다이어트 구미',
 '(직매입) 종근당건강 아이클리어 루테인']

In [222]:
mothercode_09

[100321,
 100315,
 100651,
 100654,
 100558,
 100562,
 100309,
 100055,
 100536,
 100539]

### 두 가지 기준 공통

In [255]:
# 상품명
popular_name_9 = list(set(name_li_9) & set(name_li_09))

In [224]:
# 마더코드
set(mothercode_09) & set(mothercode_9)

{100055, 100309, 100315, 100321, 100539, 100558, 100562, 100651, 100654}

## 데이터프레임에서 판매 단가 비교
> 위에서 구한 상품군별 인기 상품을 데이터프레임 내에서 탐색하고, 판매단가를 비교하여 최적 상품을 찾아본다.

In [256]:
filter_df9 = filter_dataframe_for_list(load_df, '상품명', popular_name_9)
filter_df9.head(10)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,성별 포함,년,월,일자,시,분,초
3545,2019-02-07 15:00:00,20.0,100321,201023,통째로 착즙한 루비 석류즙 84포,9,77000,23616000,0,2019,2,7,15,0,0
3546,2019-02-07 15:20:00,20.0,100321,201023,통째로 착즙한 루비 석류즙 84포,9,77000,43912000,0,2019,2,7,15,20,0
3547,2019-02-07 15:40:00,20.0,100321,201023,통째로 착즙한 루비 석류즙 84포,9,77000,57027000,0,2019,2,7,15,40,0
3755,2019-02-09 18:20:00,20.0,100321,201023,통째로 착즙한 루비 석류즙 84포,9,77000,26326000,0,2019,2,9,18,20,0
3756,2019-02-09 18:40:00,20.0,100321,201023,통째로 착즙한 루비 석류즙 84포,9,77000,66508000,0,2019,2,9,18,40,0
4324,2019-02-15 14:00:00,20.0,100321,201023,통째로 착즙한 루비 석류즙 84포,9,77000,24872000,0,2019,2,15,14,0,0
4325,2019-02-15 14:20:00,20.0,100321,201023,통째로 착즙한 루비 석류즙 84포,9,77000,38785000,0,2019,2,15,14,20,0
4326,2019-02-15 14:40:00,20.0,100321,201023,통째로 착즙한 루비 석류즙 84포,9,77000,35292000,0,2019,2,15,14,40,0
4857,2019-02-20 12:00:00,20.0,100321,201023,통째로 착즙한 루비 석류즙 84포,9,77000,17941000,0,2019,2,20,12,0,0
4858,2019-02-20 12:20:00,20.0,100321,201023,통째로 착즙한 루비 석류즙 84포,9,77000,38832000,0,2019,2,20,12,20,0


## 침구

### 취급액

In [110]:
#'침구'
df_10_sold = df_10.sort_values(by=['취급액'], ascending=False)

In [192]:
# 상품명별, 마더코드별 중복 데이터 제거
top_100_10 = index_dataframe(df_10_sold, 100)
name_li_10 = column_set_to_list(top_100_10, '상품명')
mothercode_10 = column_set_to_list(top_100_10, '마더코드')

In [193]:
name_li_10

['보몽드 엘사 자수 쿨 시어서커 침구세트 K',
 '한빛 페르시안스타일 셔닐 카페트 점보특대형',
 '보몽드 샐비어 시어서커 리플 침구세트 SS',
 '보몽드 엘사 자수 쿨 시어서커 침구세트 Q',
 '효재 화원 자수 광목워싱 침구세트 킹',
 '보몽드 엘사 자수 쿨 시어서커 침구세트 SK(슈퍼킹)',
 '보몽드 샐비어 시어서커 리플 침구세트 K',
 '보몽드 샐비어 시어서커 리플 침구세트 Q',
 '안지 청풍 대자리',
 '효재 자수 광목 워싱 침구세트 퀸',
 '보몽드 카밀라 더블착번극세사 토퍼침구세트 SK(슈퍼킹)',
 '효재 자수 광목 워싱 침구세트 킹',
 '보몽드 엘사 자수 쿨 시어서커 침구세트 SS']

In [194]:
mothercode_10

[100200, 100139, 100813, 100143, 100447]

### 주문량

In [111]:
df_10 = add_column(df_10)
df_10 = df_10.sort_values(by=['주문량'], ascending=False)
df_10

C:\Users\CUCHO\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,성별 포함,년,월,일자,시,분,초,주문량
20523,2019-07-13 14:40:00,20.000000,100447,201375,안지 청풍 대자리,10,50900,58102000,0,2019,7,13,14,40,0,1141
20522,2019-07-13 14:20:00,20.000000,100447,201375,안지 청풍 대자리,10,50900,43265000,0,2019,7,13,14,20,0,850
21112,2019-07-18 12:40:00,20.000000,100447,201375,안지 청풍 대자리,10,50900,36066000,0,2019,7,18,12,40,0,708
23267,2019-08-07 14:40:00,20.000000,100447,201375,안지 청풍 대자리,10,50900,31497000,0,2019,8,7,14,40,0,618
16268,2019-06-02 13:40:00,20.000000,100139,200373,보몽드 샐비어 시어서커 리플 침구세트 Q,10,59900,36033000,0,2019,6,2,13,40,0,601
16267,2019-06-02 13:40:00,20.000000,100139,200374,보몽드 샐비어 시어서커 리플 침구세트 SS,10,49900,29864000,0,2019,6,2,13,40,0,598
16269,2019-06-02 13:40:00,20.000000,100139,200372,보몽드 샐비어 시어서커 리플 침구세트 K,10,69900,38399000,0,2019,6,2,13,40,0,549
16265,2019-06-02 13:20:00,20.000000,100139,200373,보몽드 샐비어 시어서커 리플 침구세트 Q,10,59900,31078000,0,2019,6,2,13,20,0,518
16266,2019-06-02 13:20:00,20.000000,100139,200372,보몽드 샐비어 시어서커 리플 침구세트 K,10,69900,35278000,0,2019,6,2,13,20,0,504
18087,2019-06-19 15:40:00,20.000000,100139,200373,보몽드 샐비어 시어서커 리플 침구세트 Q,10,59900,29628000,0,2019,6,19,15,40,0,494


In [195]:
# 상품명별, 마더코드별 중복 데이터 제거
top_100_010 = index_dataframe(df_10, 100)
name_li_010 = column_set_to_list(top_100_010, '상품명')
mothercode_010 = column_set_to_list(top_100_010, '마더코드')

In [196]:
name_li_010

['보몽드 엘사 자수 쿨 시어서커 침구세트 K',
 '보몽드 샐비어 시어서커 리플 침구세트 SS',
 '한빛 페르시안스타일 셔닐 카페트 점보특대형',
 '보몽드 엘사 자수 쿨 시어서커 침구세트 Q',
 '보몽드 엘사 자수 쿨 시어서커 침구세트 SK(슈퍼킹)',
 '보몽드 샐비어 시어서커 리플 침구세트 K',
 '보몽드 샐비어 시어서커 리플 침구세트 Q',
 '안지 청풍 대자리',
 '보몽드 카밀라 더블착번극세사 토퍼침구세트 SS(슈퍼싱글)',
 '보몽드 엘사 자수 쿨 시어서커 침구세트 SS']

In [197]:
mothercode_010

[100143, 100139, 100813, 100447]

### 두 기준 공통

In [258]:
# 상품명
popular_name_10 = list(set(name_li_010) & set(name_li_10))

In [259]:
# 마더코드
set(mothercode_010) & set(mothercode_10)

{100139, 100143, 100447, 100813}

## 데이터프레임에서 판매 단가 비교
> 위에서 구한 상품군별 인기 상품을 데이터프레임 내에서 탐색하고, 판매단가를 비교하여 최적 상품을 찾아본다.

In [260]:
filter_df10 = filter_dataframe_for_list(load_df, '상품명', popular_name_10)
filter_df10.head(10)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,성별 포함,년,월,일자,시,분,초
18813,2019-06-27 12:00:00,20.0,100139,200390,보몽드 엘사 자수 쿨 시어서커 침구세트 K,10,79900,6775000,0,2019,6,27,12,0,0
18817,2019-06-27 12:20:00,20.0,100139,200390,보몽드 엘사 자수 쿨 시어서커 침구세트 K,10,79900,10001000,0,2019,6,27,12,20,0
18821,2019-06-27 12:40:00,20.0,100139,200390,보몽드 엘사 자수 쿨 시어서커 침구세트 K,10,79900,18527000,0,2019,6,27,12,40,0
19112,2019-06-30 12:00:00,20.0,100139,200390,보몽드 엘사 자수 쿨 시어서커 침구세트 K,10,79900,5676000,0,2019,6,30,12,0,0
19116,2019-06-30 12:20:00,20.0,100139,200390,보몽드 엘사 자수 쿨 시어서커 침구세트 K,10,79900,14100000,0,2019,6,30,12,20,0
19120,2019-06-30 12:40:00,20.0,100139,200390,보몽드 엘사 자수 쿨 시어서커 침구세트 K,10,79900,28581000,0,2019,6,30,12,40,0
19215,2019-07-01 12:00:00,20.0,100139,200390,보몽드 엘사 자수 쿨 시어서커 침구세트 K,10,79900,1824000,0,2019,7,1,12,0,0
19219,2019-07-01 12:20:00,20.0,100139,200390,보몽드 엘사 자수 쿨 시어서커 침구세트 K,10,79900,10347000,0,2019,7,1,12,20,0
19223,2019-07-01 12:40:00,20.0,100139,200390,보몽드 엘사 자수 쿨 시어서커 침구세트 K,10,79900,12736000,0,2019,7,1,12,40,0
19542,2019-07-04 11:00:00,20.0,100139,200390,보몽드 엘사 자수 쿨 시어서커 침구세트 K,10,79900,4473000,0,2019,7,4,11,0,0


# 상품별 매출 최적 시간대 탐색
> 위에서 '취급액'과 '주문량' 기준으로 상품군별 인기상품을 분류해 보았다. 하지만 어떤 상품이라도 한 시간대에만 방송에 노출되지는 않는다. 상품별로 그 상품이 가장 잘 팔리는 시간대가 존재한다. 인기상품별로 그 시간대를 탐색하여 편성 계획에 반영해보자.

In [315]:
# 각 인기 상품 데이터프레임 병합(효율성 고려)
filter_li = [filter_df0, filter_df1, filter_df2, filter_df3, 
             filter_df4, filter_df5, filter_df6, filter_df7, 
             filter_df8, filter_df9, filter_df10]
popular_product_df = pd.concat(filter_li)
popular_product_df.head(10)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,성별 포함,년,월,일자,시,분,초
37228,2019-12-20 08:20:00,20.0,100264,200878,셀렉프로 뚝배기 전기밥솥(멀티쿠커),0,79000,29415000,0,2019,12,20,8,20,0
37229,2019-12-20 08:40:00,20.0,100264,200878,셀렉프로 뚝배기 전기밥솥(멀티쿠커),0,79000,58490000,0,2019,12,20,8,40,0
37230,2019-12-20 09:00:00,20.0,100264,200878,셀렉프로 뚝배기 전기밥솥(멀티쿠커),0,79000,85353000,0,2019,12,20,9,0,0
37960,2019-12-28 08:20:00,20.0,100264,200878,셀렉프로 뚝배기 전기밥솥(멀티쿠커),0,79000,46738000,0,2019,12,28,8,20,0
37961,2019-12-28 08:40:00,20.0,100264,200878,셀렉프로 뚝배기 전기밥솥(멀티쿠커),0,79000,86252000,0,2019,12,28,8,40,0
37962,2019-12-28 09:00:00,20.0,100264,200878,셀렉프로 뚝배기 전기밥솥(멀티쿠커),0,79000,128382000,0,2019,12,28,9,0,0
16784,2019-06-07 08:00:00,30.0,100478,201452,파뷔에 항균도마 1+1+칼세트,0,49800,36015000,0,2019,6,7,8,0,0
16785,2019-06-07 08:30:00,30.0,100478,201452,파뷔에 항균도마 1+1+칼세트,0,49800,70355000,0,2019,6,7,8,30,0
17106,2019-06-10 09:00:00,30.0,100478,201452,파뷔에 항균도마 1+1+칼세트,0,49800,35161000,0,2019,6,10,9,0,0
17107,2019-06-10 09:30:00,30.0,100478,201452,파뷔에 항균도마 1+1+칼세트,0,49800,57050000,0,2019,6,10,9,30,0


In [328]:
# 상품명 기준으로 데이터 분리
df_li = []
total_popular_name = list(set(popular_product_df['상품명']))
mod=sys.modules[__name__]

for i in range(len(total_popular_name)):
    setattr(mod, 'product_df_{}'.format(i), load_df[load_df['상품명']==total_popular_name[i]][:1]) # 전체 122개 

In [342]:
# 상품별 데이터프레임 모으기
raw_li = []
for i in range(123):
    raw_li.append('product_df_{}'.format(i))

def dataframe_concat(df1, df2):
    df1 = pd.concat([df1, df2])
    return df1

In [345]:
# 데이터 합치기
for i in range(1, 123):
    product_df_0 = dataframe_concat(product_df_0, eval(raw_li[i]))

In [346]:
# 상품군별 취합 완료
product_df_0

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,성별 포함,년,월,일자,시,분,초
29289,2019-10-03 14:00:00,20.0,100161,200566,시스마르스 플렉시블 웨지 펌프스,4,29800,45927000,0,2019,10,3,14,0,0
18812,2019-06-27 12:00:00,20.0,100139,200391,보몽드 엘사 자수 쿨 시어서커 침구세트 Q,10,69900,5667000,0,2019,6,27,12,0,0
19207,2019-07-01 10:00:00,20.0,100444,201369,쿠미투니카 코르셋 하이웨스트 팬티세트,3,59900,23865000,0,2019,7,1,10,0,0
29242,2019-10-03 08:00:00,20.0,100248,200830,[K-SWISS] 19 F/W 남성 패딩 트랙수트 2종,2,79900,10934000,1,2019,10,3,8,0,0
12600,2019-05-02 01:00:00,20.0,100348,201090,벨레즈온 브라캡 심리스 원피스 패키지,3,49900,16511000,2,2019,5,2,1,0,0
14548,2019-05-17 13:00:00,20.0,100139,200373,보몽드 샐비어 시어서커 리플 침구세트 Q,10,59900,7460000,0,2019,5,17,13,0,0
267,2019-01-04 06:00:00,20.0,100651,201972,제주농장 유기 양배추진액 120포,9,74900,10439000,0,2019,1,4,6,0,0
361,2019-01-05 08:00:00,20.0,100761,202216,마담팰리스 웜기모 롱베스트 3종세트,2,39900,16654000,0,2019,1,5,8,0,0
32425,2019-11-03 09:20:00,30.0,100091,200248,(1등급)221L_딤채김치냉장고,1,899000,5028000,0,2019,11,3,9,20,0
25699,2019-08-31 06:00:00,20.0,100809,202384,CERINI by PAT 남성 어반 카라 티셔츠 4종(8월),2,49900,17542000,1,2019,8,31,6,0,0


In [356]:
# 상품군별 정렬
product_df_0 = product_df_0.sort_values(by='상품군', ascending=True)

# 중복 column 제거
product_df = product_df_0[['방송일시', '노출(분)', '마더코드', '상품코드', '상품명', '상품군', '판매단가', '취급액', '성별 포함']]

In [377]:
# 병합된 데이터 확인
product_df.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,성별 포함
16784,2019-06-07 08:00:00,30.0,100478,201452,파뷔에 항균도마 1+1+칼세트,0,49800,36015000,0
21,2019-01-01 12:00:00,20.0,100088,200236,에코라믹 통주물 스톤 냄비세트,0,60900,33823000,0
12634,2019-05-02 12:00:00,20.0,100475,201446,초특가 파뷔에 팬지 대형 후라이팬 세트,0,59800,20982000,0
28406,2019-09-25 06:00:00,20.0,100724,202115,IH 옛 가마솥 세트,0,62000,9761000,0
198,2019-01-03 14:00:00,20.0,100255,200868,한일 대용량 스텐 분쇄믹서기,0,109800,22816000,0
37833,2019-12-26 19:20:00,20.0,100638,201956,램프쿡 자동회전냄비,0,109000,32379000,0
9609,2019-04-06 07:00:00,20.0,100478,201453,파뷔에 항균도마 1+1세트,0,49800,27033000,0
37228,2019-12-20 08:20:00,20.0,100264,200878,셀렉프로 뚝배기 전기밥솥(멀티쿠커),0,79000,29415000,0
45,2019-01-01 20:00:00,20.0,100150,200533,일시불 LG 통돌이 세탁기,1,439000,35519000,0
35040,2019-11-29 22:20:00,20.0,100148,200412,LG 울트라HD TV 65UM7900BNA,1,1700000,37696000,0


In [358]:
# 데이터로 저장
product_df.to_csv('인기 상품 시간대 데이터.csv', encoding='cp949')